# Imports

In [49]:
import os
os.system('pip install tasknet pygrove xpflow setGPU pandit imblearn transformers==4.28.0 wandb')

0

In [50]:
import pandit as pd
import numpy as np
import tasknet as tn
from tqdm.notebook import tqdm
from dataclasses import dataclass
tqdm.pandas()
import random
from datasets import DatasetDict
import datasets
from torch.utils.data import Dataset
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoTokenizer
import inspect
import torch
import transformers
from types import MethodType
os.environ["WANDB_DISABLED"] = "false"
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

import tensorflow as tf
from collections import Counter
import imblearn
import funcy as fc

import wandb

In [51]:
wandb.login()

wandb: Currently logged in as: quentintilman (master_thesis_2023). Use `wandb login --relogin` to force relogin


True

In [ ]:
# import pkg_resources
# pkg_resources.require("transformers==4.28.0")
# import transformers

In [52]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [53]:
from huggingface_hub import login
from  credentials import huggingfacehub #create credentials.py file with huggingfacehub variable and API key as value.
login(huggingfacehub)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to C:\Users\QT\.huggingface\token
Login successful


# Loading Data

## Datasets

In [54]:
url = 'https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/RedditPoll350K/RedditPoll350K_categorized.json'
df_reddit= pd.read_json('https://sileod.s3.us-west-004.backblazeb2.com/r_poll_all.json')

In [55]:
df = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/WVMCQ7/WVMCQ7-Normalized_labels_percentage_wise.json")
df = df[df["demographic_topic"]!= "politic"] 

## Significant P-value subset

In [56]:
redditPoll350k_significant = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/RedditPoll350K/RedditPoll350K_significant_divisiveness_questions.json")
redditPoll350k_significant_reduced = redditPoll350k_significant#[redditPoll350k_significant["p_value"]< 1e-4]

## Demographic libraries

In [61]:
stop_words = ["able","about","above","abroad","according","accordingly","across","actually","adj","after","afterwards","again","against","ago","ahead","ain't","all","allow","allows","almost","alone","along","alongside","already","also","although","always","am","amid","amidst","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","a's","aside","ask","asking","associated","at","available","away","awfully","back","backward","backwards","be","became","because","become","becomes","becoming","been","before","beforehand","begin","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","came","can","cannot","cant","can't","caption","cause","causes","certain","certainly","changes","clearly","c'mon","co","co.","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","c's","currently","dare","daren't","definitely","described","despite","did","didn't","different","directly","do","does","doesn't","doing","done","don't","down","downwards","during","each","edu","eg","eight","eighty","either","else","elsewhere","end","ending","enough","entirely","especially","et","etc","even","ever","evermore","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","fairly","far","farther","few","fewer","fifth","first","five","followed","following","follows","for","forever","former","formerly","forth","forward","found","four","from","further","furthermore","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","had","hadn't","half","happens","hardly","has","hasn't","have","haven't","having","he","he'd","he'll","hello","help","hence","her","here","hereafter","hereby","herein","here's","hereupon","hers","herself","he's","hi","him","himself","his","hither","hopefully","how","howbeit","however","hundred","i'd","ie","if","ignored","i'll","i'm","immediate","in","inasmuch","inc","inc.","indeed","indicate","indicated","indicates","inner","inside","insofar","instead","into","inward","is","isn't","it","it'd","it'll","its","it's","itself","i've","just","k","keep","keeps","kept","know","known","knows","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","likewise","little","look","looking","looks","low","lower","ltd","made","mainly","make","makes","many","may","maybe","mayn't","me","mean","meantime","meanwhile","merely","might","mightn't","mine","minus","miss","more","moreover","most","mostly","mr","mrs","much","must","mustn't","my","myself","name","namely","nd","near","nearly","necessary","need","needn't","needs","neither","never","neverf","neverless","nevertheless","new","next","nine","ninety","no","nobody","non","none","nonetheless","noone","no-one","nor","normally","not","nothing","notwithstanding","novel","now","nowhere","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","one's","only","onto","opposite","or","other","others","otherwise","ought","oughtn't","our","ours","ourselves","out","outside","over","overall","own","particular","particularly","past","per","perhaps","placed","please","plus","possible","presumably","probably","provided","provides","que","quite","qv","rather","rd","re","really","reasonably","recent","recently","regarding","regardless","regards","relatively","respectively","right","round","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","shan't","she","she'd","she'll","she's","should","shouldn't","since","six","so","some","somebody","someday","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","take","taken","taking","tell","tends","th","than","thank","thanks","thanx","that","that'll","thats","that's","that've","the","their","theirs","them","themselves","then","thence","there","thereafter","thereby","there'd","therefore","therein","there'll","there're","theres","there's","thereupon","there've","these","they","they'd","they'll","they're","they've","thing","things","think","third","thirty","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","till","to","together","too","took","toward","towards","tried","tries","truly","try","trying","t's","twice","two","un","under","underneath","undoing","unfortunately","unless","unlike","unlikely","until","unto","up","upon","upwards","us","use","used","useful","uses","using","usually","v","value","various","versus","very","via","viz","vs","want","wants","was","wasn't","way","we","we'd","welcome","well","we'll","went","were","we're","weren't","we've","what","whatever","what'll","what's","what've","when","whence","whenever","where","whereafter","whereas","whereby","wherein","where's","whereupon","wherever","whether","which","whichever","while","whilst","whither","who","who'd","whoever","whole","who'll","whom","whomever","who's","whose","why","will","willing","wish","with","within","without","wonder","won't","would","wouldn't","yes","yet","you","you'd","you'll","your","you're","yours","yourself","yourselves","you've","zero","a","how's","i","when's","why's","b","c","d","e","f","g","h","j","l","m","n","o","p","q","r","s","t","u","uucp","w","x","y","z","I","www","amount","bill","bottom","call","computer","con","couldnt","cry","de","describe","detail","due","eleven","empty","fifteen","fifty","fill","find","fire","forty","front","full","give","hasnt","herse","himse","interest","itse”","mill","move","myse”","part","put","show","side","sincere","sixty","system","ten","thick","thin","top","twelve","twenty","abst","accordance","act","added","adopted","affected","affecting","affects","ah","announce","anymore","apparently","approximately","aren","arent","arise","auth","beginning","beginnings","begins","biol","briefly","ca","date","ed","effect","et-al","ff","fix","gave","giving","heres","hes","hid","home","id","im","immediately","importance","important","index","information","invention","itd","keys","kg","km","largely","lets","line","'ll","means","mg","million","ml","mug","na","nay","necessarily","nos","noted","obtain","obtained","omitted","ord","owing","page","pages","poorly","possibly","potentially","pp","predominantly","present","previously","primarily","promptly","proud","quickly","ran","readily","ref","refs","related","research","resulted","resulting","results","run","sec","section","shed","shes","showed","shown","showns","shows","significant","significantly","similar","similarly","slightly","somethan","specifically","state","states","stop","strongly","substantially","successfully","sufficiently","suggest","thered","thereof","therere","thereto","theyd","theyre","thou","thoughh","thousand","throug","til","tip","ts","ups","usefully","usefulness","'ve","vol","vols","wed","whats","wheres","whim","whod","whos","widely","words","world","youd","youre"]
import requests
import re

r = requests.get("https://raw.githubusercontent.com/dariusk/corpora/master/data/religion/religions.json")
religions = r.json()
beliefs = []
for major in [item for item in religions.values()]:
    majors = [m for m in major]
    beliefs.extend(majors)
    try:
        for minor in major.values():
            for religion in minor:
                beliefs.append(religion)
    except:
        next
r = requests.get("https://raw.githubusercontent.com/Imagin-io/country-nationality-list/master/countries.json")
dictionary = r.json()
df_dictionary = pd.DataFrame(dictionary)
def nation(item):
    item["en_short_name"] = ""+item["en_short_name"].lower()+""
    item["alpha_2_code"] = ""+item["alpha_2_code"].lower()+""
    item["alpha_3_code"] = ""+item["alpha_3_code"].lower()+""
    nationality = re.split(', | or',item["nationality"].lower())
    item["nationality"] = nationality
    return item
df_dictionary=df_dictionary.apply(nation,axis=1)
nationalities = []
for item in df_dictionary["nationality"].values:
    for label in item:
        nationalities.append(label)

countries = []
countries.extend(df_dictionary["en_short_name"].values)
countries.extend(df_dictionary["alpha_2_code"].values)
countries.extend(df_dictionary["alpha_3_code"].values)

# Data selection and processing

## DataCollator

In [58]:
@dataclass
class DataCollatorForPolls:
    tokenizer: None=None
    tokenizer_kwargs: None=None
    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        #tasks = [feature.pop("task") for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)]
            for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        pad_args=inspect.signature(self.tokenizer.pad).parameters.keys()
        batch = self.tokenizer.pad(flattened_features, **fc.project(self.tokenizer_kwargs,pad_args))
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.stack(labels).float() #### Allows one value per choice
        #batch["task"] = torch.tensor(tasks, dtype=torch.int64)
        return batch

## Preprocessing Methods

In [59]:
import importlib
importlib.reload(tn)

<module 'tasknet' from 'c:\\Python35\\lib\\site-packages\\tasknet\\__init__.py'>

In [60]:
#reddit
def get_demographics(x):
    parenthesized=[s[s.find("(")+1:s.rfind(")")]  for s in x.options.keys()]
    parenthesized=[s for s in parenthesized if s]
    parenthesized_count=pd.Series.value_counts(parenthesized)
    return list(parenthesized_count[parenthesized_count>1].index)

def split_questions(x):
    demographics=get_demographics(x)

    l=[]
    for d in demographics:
        l+=[fc.project(x.options, [x for x in x.options if f'({d})' in x])] #if row has multiple demographics its repeated in dataset?
    if l:
        x.options=l
        x['split']=True
    else:
        x.options=[x.options]
        x['split']=False
    return x

def extract_demographic(x):
    demographics = get_demographics(x)
    x['demographic']=''
    if not demographics:
        return x
    if len(demographics)>1:
        print('failure:',demographics, x.options)
        return x
    demographic=demographics[0]
    x['options'] = {k.replace(f'({demographic})',''):v for (k,v) in x.options.items()}
    x['demographic']=demographic
    return x

num_choices=6
choices_names=[f'choice_{i}' for i in range(num_choices)]
mode='proba'#"binary"#

def preprocess(x):
    choices = (list(x.options.keys())+['']*num_choices)[:num_choices]
    for i,n in enumerate(choices_names):
        x[n]=choices[i]
    scores = np.array(list(x.options.values())+[-1]*num_choices)[:num_choices]*1.0
    if mode=='binary':
        x['label'] = np.random.choice(np.flatnonzero(scores == scores.max()))
    else:
        scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()
        x['label'] = scores
    del x['options']
    x['task']=0
    return x

def remove_result(x):
  newDict={}
  for key,value in x.items():
    if "result" not in key.lower():
      newDict[key] = value
  return newDict

In [62]:
def choices(x):
    choices = (x["options"]+['']*num_choices)[:num_choices]
    scores = np.array(list(x.labels)+[-1]*num_choices)[:num_choices]*1.0
    for i,n in enumerate(choices_names):
        x[n]=choices[i]  
    x["labels"] = scores
    return x

def filter_data(x):
    options = x["options"]
    if(1 < len(options) <7):
        return x

def apply_question_template(df):
  #we need to take into account that the input to the model is Demographic + Question + Answer
  # we thus need to create a question template that seems natural
  # This template is only necessary if the input is not already in a question format
  demographic = df["demographic"]
  category = df["demographic_category"]
  subject = df["input"]
  result = any(item in subject for item in ["?","How","Why","Where","When","Who","What"])
  sentence = f"{demographic}: {subject} ?".lower()
  if(not result):
    if(category == "age"):
        sentence = f"{demographic} years old: which of these options would you take in regard to the subject '{subject}'?".lower()  
    elif(category == "married"):
        sentence = f"relationship satus {demographic}: which of these options would you take in regard to the subject '{subject}'?".lower()
    else:
        sentence = f"{demographic}: which of these options would you take in regard to the subject '{subject}'?".lower()  
  df['question'] = sentence
  return df

def get_split(x,train,validation):
    """parameters are wished percentage of dataset to be given to a split"""
    rnd=random.Random(x).random()
    if rnd<train:
        return 'train'
    if rnd<(train + validation):
        return 'validation'
    else:
        return 'test'
        
def to_year(x):
    x["created"] = pd.to_datetime(x["created"], unit='s').year
    return x

def shuffle_options(x):
    options = x["options"]
    labels = x["labels"]
    tupples = list(zip(options,labels))
    random.shuffle(tupples)
    x["options"] = [k for (k,_) in tupples]
    x["labels"] = [v for (_,v) in tupples]
    return x

## RedditPoll350k

In [63]:
df_reddit['inputs']=df_reddit.title
len(df_reddit)

312306

In [64]:
df_reddit['options']=df_reddit.options.map(lambda x:{k:v for k,v in x.items() if k!='Results'})
df_reddit['options'].sample(frac=1.0)#mix options ad random
df_reddit=df_reddit[df_reddit['options'].map(len)>1]
len(df_reddit)
df_reddit['n_answers']=df_reddit.options.map(dict.values).map(sum)
df_reddit=df_reddit[df_reddit.n_answers>20]#filter on number of minimum answers !!! 
len(df_reddit)

307568

In [65]:
if 'dem' not in url:
    df_reddit=df_reddit.progress_apply(split_questions,axis=1).explode('options')
    df_reddit=df_reddit.progress_apply(extract_demographic,axis=1)
df_reddit_demographics=df_reddit[df_reddit.demographic.map(len).map(bool)]
df_reddit_demographics.head(3), len(df_reddit_demographics)

  0%|          | 0/307568 [00:00<?, ?it/s]

  0%|          | 0/317731 [00:00<?, ?it/s]

failure: ['183 cm', "165 cm) - 6' (183 cm"] {'5\' 5" (165 cm) - 6\' (183 cm), Male': 106, '5\' 5" (165 cm) - 6\' (183 cm), Female': 36, "> 6' (183 cm), Male": 65, "> 6' (183 cm), Female": 4}
failure: ["165 cm) - 6' (183 cm", '165 cm'] {'< 5\' 5" (165 cm), Male': 14, '< 5\' 5" (165 cm), Female': 34, '5\' 5" (165 cm) - 6\' (183 cm), Male': 106, '5\' 5" (165 cm) - 6\' (183 cm), Female': 36}


(        id                                              title  \
 1   yvae7l  Do You Believe Your Congressional District is ...   
 1   yvae7l  Do You Believe Your Congressional District is ...   
 65  yuipdn                            Does body count matter?   
 
                     options     created  \
 1   {'Yes ': 35, 'No ': 17}  1668455219   
 1     {'Yes ': 7, 'No ': 8}  1668455219   
 65  {'Yes ': 70, 'No ': 52}  1668382874   
 
                                                inputs  n_answers  split  \
 1   Do You Believe Your Congressional District is ...        134   True   
 1   Do You Believe Your Congressional District is ...        134   True   
 65                            Does body count matter?        168   True   
 
    demographic  
 1            D  
 1            R  
 65        Male  ,
 20069)

In [66]:
df_reddit_demographics=df_reddit[df_reddit.demographic.map(len).map(bool)]
df_reddit_demographics.head(3), len(df_reddit_demographics)

(        id                                              title  \
 1   yvae7l  Do You Believe Your Congressional District is ...   
 1   yvae7l  Do You Believe Your Congressional District is ...   
 65  yuipdn                            Does body count matter?   
 
                     options     created  \
 1   {'Yes ': 35, 'No ': 17}  1668455219   
 1     {'Yes ': 7, 'No ': 8}  1668455219   
 65  {'Yes ': 70, 'No ': 52}  1668382874   
 
                                                inputs  n_answers  split  \
 1   Do You Believe Your Congressional District is ...        134   True   
 1   Do You Believe Your Congressional District is ...        134   True   
 65                            Does body count matter?        168   True   
 
    demographic  
 1            D  
 1            R  
 65        Male  ,
 20069)

In [67]:
counter = [x for x in Counter(df_reddit_demographics["demographic"]).keys() if Counter(df_reddit_demographics["demographic"])[x] >= 10]
df_reddit_demographics= df_reddit_demographics[df_reddit_demographics["demographic"].isin(counter)]

In [68]:
df_reddit_demographics = df_reddit_demographics.progress_apply(preprocess,axis=1)

  0%|          | 0/11561 [00:00<?, ?it/s]

<ipython-input-60-ee9b09971a0f>:47: RuntimeWarning: invalid value encountered in true_divide
  scores[scores!=-1]= scores[scores!=-1]/scores[scores!=-1].sum()


In [69]:
if mode=='binary':
    df_reddit_demographics=df_reddit_demographics[df_reddit_demographics.label<num_choices]
else:
    df_reddit_demographics=df_reddit_demographics[df_reddit_demographics.label.map(len)==num_choices]

len(df_reddit_demographics)

11561

In [70]:
df_reddit_demographics = df_reddit_demographics.dropna()

In [71]:
if mode=="binary":
  df_reddit_demographics[df_reddit_demographics.label.progress_map(lambda x: np.isnan(x))]
else:
  df_reddit_demographics[df_reddit_demographics.label.progress_map(lambda x: any(np.isnan(a) for a in x))]

  0%|          | 0/11561 [00:00<?, ?it/s]

### Normalize demographic

In [72]:
def normalize_nationality(x):
    male = ["man","guy"]
    label = x["demographic"].lower()
    if (2 <= len(label) <= 3) and label not in male:
      if any(token.replace(" ","").lower() in label.split(" ") for token in nationalities):
        token = [token for token  in nationalities if token.replace(" ","").lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
              label = "not "+token[0]
        else:
              label = token[0]
    elif any(token.lower() in label.split(" ") for token in nationalities):
        token = [token for token  in nationalities if token.lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
    x["demographic"] = label
    return x

def normalize_countries(x):
    male = ["man","guy","men"]
    label = x["demographic"].lower()
    if (2 <= len(label) <= 3) and label not in male:
      if any(token.replace(" ","").lower() in label.split(" ") for token in countries):
        token = [token for token  in countries if token.replace(" ","").lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
              label = "not "+token[0]
        else:
              label = token[0]
    elif any(token.lower() in label.split(" ") for token in countries):
        token = [token for token  in countries if token.lower() in label.split(" ")]
        if("not" in label) | ("non" in label):
            label = "not "+token[0]
        else:
            label = token[0]
    x["demographic"] = label
    return x

def normalize_genders(x):
    label = x["demographic"].lower()
    female= ["f","female","woman","chick","girl"]
    male = ["m","male","man","guy","men"]
    if any(token.lower() in female for token in label.split()):
        if ("not" in label) | ("non" in label):
            label = "not female"
        else:
            label = "female"
    if any(token in male for token in label.split()):
        if("not" in label) | ("non" in label):
            label = "not male"
        else:
            label = "male"
    x["demographic"] = label
    return x

def normalize_politics(x):
    label = x["demographic"].lower()
    right = ["d","democratic","right","r",'right-leaning']
    left = ["republican","republicans","left","left-leaning"]
    if any(token.lower() in label.split(" ") for token in right):
        if("not" in label) | ("non" in label):
            label = "not democrat"
        else: 
            label = "democrat"
    elif any(token.lower() in label.split(" ") for token in left):
        if("not" in label) | ("non" in label):
            label = "not republican"
        else: 
            label = "republican"
    x["demographic"] = label
    return x

def remove_nan(x):
  labels = x["label"]
  if mode == "binary":
      if np.isnan(labels):
          pass
      else:
          return x
  else:
    if any(np.isnan(labels)):
        pass
    else:
        return x


In [73]:
df_reddit_demographics_normalized_labels=df_reddit_demographics.apply(normalize_nationality,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.apply(normalize_genders,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.apply(normalize_countries,axis=1)
df_reddit_demographics_normalized_labels=df_reddit_demographics_normalized_labels.apply(normalize_politics,axis=1)

In [74]:
df_reddit_demographics_normalized_labels = df_reddit_demographics_normalized_labels.progress_apply(remove_nan,axis=1)
df_reddit_demographics_normalized_labels = df_reddit_demographics_normalized_labels.dropna()
df_reddit_demographics_normalized_labels['inputs']=df_reddit_demographics_normalized_labels.demographic+': '+df_reddit_demographics_normalized_labels.title

  0%|          | 0/11561 [00:00<?, ?it/s]

#### Fine-tuning and Zeroshot for MCQ setting

In [75]:
df_reddit_demographics_normalized_labels['inputs']=df_reddit_demographics_normalized_labels.demographic+': '+df_reddit_demographics_normalized_labels.title

In [76]:
df_reddit_demographics_normalized_labels_reduced = df_reddit_demographics_normalized_labels.drop_duplicates(
  subset = ['title', 'demographic'],
  keep = 'last').reset_index(drop = True)

### Example

In [77]:
tokenizer =AutoTokenizer.from_pretrained("roberta-large", use_fast=True)

In [78]:
features = tokenizer("Hello, this one sentence!", "And this sentence goes with it.",truncation=True)
print(features)
tokenizer.decode(features["input_ids"])

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2, 2, 2409, 42, 3645, 1411, 19, 24, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


'<s>Hello, this one sentence!</s></s>And this sentence goes with it.</s>'

### Reco

#### Fine-tuning

##### RedditPoll350k

In [79]:
df_reddit_full = df_reddit

In [80]:
df_reddit_full['split']=df_reddit_full.title.map(lambda x: get_split(x,0.95,0.025))

In [81]:
df_reddit_full = df_reddit_full[df_reddit_full["split"] == "validation"]
df_reddit_full.reset_index(drop=True)

,id,title,options,created,inputs,n_answers,split,demographic
0,yur32f,Can you both snap your fingers and whistle?,"{'Yes': 86, 'Only snap': 25, 'Only whistle': 9...",1668407624,Can you both snap your fingers and whistle?,130,validation,
1,yucvxt,"If you had the same power as Thanos, how many ...","{'0': 55, '1': 14, '2': 4, '3': 3, '4 or more'...",1668370857,"If you had the same power as Thanos, how many ...",109,validation,
2,yu58u8,What type of Women do you prefer out of these ...,"{'YouTuber': 23, 'Twitch Streamer': 17, 'Athle...",1668354238,What type of Women do you prefer out of these ...,353,validation,
3,yu3hva,Are Indians (Country) Asians in your book?,"{'Yes': 92, 'No, they are separate from Asians...",1668350386,Are Indians (Country) Asians in your book?,114,validation,
4,ytuk6v,Which cookie are you nostalgic about?,"{'“Mother’s cookies”': 12, 'Those hard vanilla...",1668323545,Which cookie are you nostalgic about?,62,validation,
...,...,...,...,...,...,...,...,...
7957,frnq8r,What do you think about King Arthur: Legend of...,"{'Awesome': 5, 'Meh!': 8, 'Good': 3, 'Bad!': 7}",1585561017,What do you think about King Arthur: Legend of...,23,validation,
7958,frccus,"Help us settle a quarantine argument, what col...","{'Pink': 172, 'Purple': 67}",1585512617,"Help us settle a quarantine argument, what col...",239,validation,
7959,fr04oe,What-If Hypothetical Poll Series (#2) [VRMMO],"{'You see a bustling village ahead of you, and...",1585457450,What-If Hypothetical Poll Series (#2) [VRMMO],47,validation,
7960,fqt3bw,How do you think Epstein died?,"{'Killed by powerful people': 77, 'Actually di...",1585431260,How do you think Epstein died?,95,validation,


In [82]:
df_reddit_full = df_reddit_full.progress_apply(preprocess,axis=1)

  0%|          | 0/7962 [00:00<?, ?it/s]

In [83]:
df_reddit_full[df_reddit_full.label.progress_map(lambda x: any(np.isnan(a) for a in x))]

  0%|          | 0/7962 [00:00<?, ?it/s]

,id,title,created,inputs,n_answers,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task


In [84]:
df_reddit_full=df_reddit_full[df_reddit_full.label.map(len)==num_choices]
len(df_reddit_full)

7962

In [85]:
def remove_empty(x):
  choiceA = x["choice_0"]
  if choiceA != '':
    return x

In [86]:
df_reddit_full_no_empty = df_reddit_full.progress_apply(remove_empty,axis=1)
df_reddit_full = df_reddit_full_no_empty.dropna()

  0%|          | 0/7962 [00:00<?, ?it/s]

In [87]:
try:
    del df_reddit_full['options']
except:
    pass
dataset_reddit_plbrt=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_full) for k in {'train','test','validation'}})

In [88]:
reco = tn.MultipleChoice(
    dataset_reddit_plbrt,
    s1="inputs",
    y="label",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_reddit_plbrt = [reco]

##### RedditPoll350k (demographic subset)

In [89]:
counts = Counter(df_reddit_demographics_normalized_labels["demographic"])
df_reddit_demographics_normalized_labels_reduced = pd.DataFrame([item for item in df_reddit_demographics_normalized_labels_reduced.values if counts[item[6]] >= 10],columns = df_reddit_demographics_normalized_labels.columns)

In [90]:
df_reddit_demographics_normalized_labels_reduced["demographic"].value_counts().keys()

Index(['male', 'female', 'american', 'other', 'not american', 'non-american',
       'democrat', 'republican', 'european', 'us', 'usa', 'religious',
       'under 18', 'women', 'center', 's', 'non-binary', 'europe', 'canadian',
       'british', '18+', 'white', 'uk', 'comment', 'rest of the world',
       'conservative', 'explain', 'atheist', 'not religious', 'nb', 'black',
       'liberal', 'no', 'christian', 'why', 'yes', 'why?', 'over 18', '<18',
       'adult', 'not uk', 'non-religious', 'america', 'north america',
       'i'm religious', 'asian', 'right-handed', 'australian', 'boy',
       'explain why', 'am', 'non binary', 'left-handed', 'indian', '30+',
       'not usa', 'non-european', '2021', 'i'm not religious', 'straight',
       'centrist', 'non-usa', 'right-wing', 'gen z', 'not am', 'not us'],
      dtype='object')

In [91]:
df_reddit_demographics_normalized_labels_reduced = df_reddit_demographics_normalized_labels_reduced.undersample(column="demographic")

In [92]:
df_reddit_demographics_normalized_labels_reduced['split']=df_reddit_demographics_normalized_labels_reduced.title.map(lambda x: get_split(x,0.975,0.025))

In [93]:
df_reddit_demographics_normalized_labels_reduced['input_ids'] = 0
df_reddit_demographics_normalized_labels_reduced['attention_mask'] = 0

In [94]:
try:
    del df_reddit_demographics_normalized_labels_reduced['options']
except:
    pass
df_reddit_demographics_normalized_labels_reduced['labels']=df_reddit_demographics_normalized_labels_reduced['label']
del df_reddit_demographics_normalized_labels_reduced['label']
# df_reddit_demographics_normalized_labels['input_ids'] = 0
# df_reddit_demographics_normalized_labels['attention_mask'] = 0
dataset_reddit_demographics_plbrt=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_demographics_normalized_labels_reduced[df_reddit_demographics_normalized_labels_reduced['split']==k]) for k in {'train','test','validation'}})

In [95]:
reco = tn.MultipleChoice(
    dataset_reddit_demographics_plbrt,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_reddit_demographics_plbrt = [reco]

### Zeroshot

##### RedditPoll350k (full)

In [96]:
df_reddit_zeroshot = pd.DataFrame(tasks_reddit_plbrt[0].dataset["validation"])

##### RedditPoll350k (Demographic subset)

In [97]:
df_demo_reddit_zeroshot = tasks_reddit_demographics_plbrt[0].dataset["validation"]

#### Show Example

In [98]:
def show_one(example):
    print(f"Context: {example['inputs']}")
    print(f"  A - {example['inputs']} {example['choice_0']}")
    print(f"  B - {example['inputs']} {example['choice_1']}")

In [99]:
show_one(dataset_reddit_demographics_plbrt["train"][0])

Context: non-usa: Do you think the current inflation rate is a problem?
  A - non-usa: Do you think the current inflation rate is a problem? Yes 
  B - non-usa: Do you think the current inflation rate is a problem? No 


## WVMCQ7

### Filtering out non relevant questions

From the questionnaires we discard questions that do not serve to capture the opinion of people. Questions about family situation, sex, age etc of a demographic is not useable for our purpose.

In [100]:
dict_questions_discard = ["Religious person",
"Neighbours: Drug addicts",
"Neighbours: People of a different race",
"Neighbours: Immigrants/foreign workers",
"Neighbours: Homosexuals",
"Neighbours: Heavy drinkers",
"Sex",
"Year of birth",
"Age",
"Respondent immigrant",
"Mother immigrant",
"Father immigrant",
"Country of birth: Respondent",
"Country of birth: Mother of the respondent",
"Country of birth: Father of the respondent",
"Citizen of [country]",
"Number of people in household",
"Do you live with your parents",
"Marital status",
"How many children do you have",
"Highest educational level attained - Respondent",
"Education level (recoded)",
"Highest educational level attained - Respondent´s Spouse",
"Highest educational level attained - Respondent´s Spouse (Recoded)",
"Highest educational level attained - Respondent´s Mother",
"Highest educational level attained - Respondent´s Mother (Recoded)",
"Highest educational level attained - Respondent´s Father",
"Highest educational level attained - Respondent´s Father (Recoded)",
"Employment status",
"Pre-school child suffers with working mother",
"Employment status - Responent´s Spouse",
"Respondent - Occupational group",
"Respondent´s Spouse - Occupational group",
"Respondent´s Father - Occupational group (when respondent was 14 years old)",
"Sector of employment",
"Scale of incomes",
"State of health (subjective)",
]

In [101]:
def discard_questions(x):
    if(x['input'] in dict_questions_discard):
        pass
    else:
        return x

In [102]:
df_filtered = df.progress_apply(discard_questions,axis= 1)
df_filtered = df_filtered.dropna()

  0%|          | 0/110901 [00:00<?, ?it/s]

In [103]:
df_filtered = df_filtered.progress_apply(filter_data,axis=1)
df_filtered = df_filtered.dropna()
pd.DataFrame(df_filtered)

  0%|          | 0/103602 [00:00<?, ?it/s]

,year,demographic,input,demographic_topic,options,labels
0,2017.0,-1,Aims of country: first choice,age,"[A high level of economic growth, Making sure ...","[0.25, 0.25, 0.25, 0.25]"
1,2017.0,-1,Aims of country: second choice,age,[Making sure this country has strong defense f...,"[0.2, 0.8]"
2,2017.0,-1,Aims of respondent: first choice,age,"[maintaining order in the nation, Giving peopl...","[0.6000000000000001, 0.2, 0.2]"
3,2017.0,-1,Aims of respondent: second choice,age,[Giving people more say in important governmen...,"[0.5, 0.25, 0.25]"
4,2017.0,-1,Believe in: God,age,"[No, Yes]","[0.5, 0.5]"
...,...,...,...,...,...,...
209537,2022.0,male,Vote in elections: local level,gender,"[Always, Usually, Never, Not allowed to vote]","[0.5963541667000001, 0.24804687500000003, 0.14..."
209538,2022.0,male,Which party would you vote for: first choice (...,gender,"[No right to vote, Other, Independent Candidate]","[0.36250000000000004, 0.125, 0.5125000000000001]"
209539,2022.0,male,Willingness to fight for country,gender,"[no, yes]","[0.22697899840000002, 0.7730210016]"
209540,2022.0,male,Work is a duty towards society,gender,"[Strongly agree, Agree, Neither agree nor disa...","[0.3264913406, 0.3688261706, 0.1610006414, 0.1..."


In [104]:
df_filtered= pd.DataFrame([item for item in df_filtered.values if item[1].lower() not in ["-5","-4","-3","-2","-1","no answer","don't know","missing","other"] ],columns= ["year","demographic","input","demographic_category","options","labels"])

In [105]:
df_filtered = df_filtered[df_filtered["year"]>2019]

In [ ]:
df_filtered['split']=df_filtered.input.progress_map(lambda x: get_split(x,0.80,0.20))

  0%|          | 0/39731 [00:00<?, ?it/s]

### Undersampling for dataset demographic attribute balancing on training set

In [ ]:
newdf = df_filtered.sample(frac=1).reset_index(drop=True).drop_duplicates(
  subset = ['input', 'demographic'],
  keep = 'last').reset_index(drop = True)

In [ ]:
df_filtered_undersampled = newdf.undersample(column="demographic")

In [ ]:
df_filtered_undersampled.progress_apply(shuffle_options,axis=1)

  0%|          | 0/15228 [00:00<?, ?it/s]

,year,demographic,input,demographic_category,options,labels,split
0,2020.0,17,"Member: Belong to conservation, the environmen...",age,"[mentioned, not mentioned]","[0.3142857143, 0.6857142857]",validation
1,2021.0,48,Confidence: Armed Forces,age,"[A great deal , None at all, Quite a lot, Not ...","[0.32, 0.11, 0.32, 0.25]",train
2,2022.0,48,Justifiable: Death penalty,age,"[Never justifiable, Always justifiable]","[0.7288135593, 0.27118644070000003]",train
3,2020.0,Single,Protecting environment vs. Economic growth,married,[Protecting the environment should be given pr...,"[0.6173297966, 0.35349248450000004, 0.02917771...",train
4,2021.0,32,Satisfaction with your life,age,"[Don't know, completely satisfied]","[0.2793296089, 0.7206703911]",train
...,...,...,...,...,...,...,...
15223,2020.0,62,Aims of respondent: first choice,age,"[maintaining order in the nation, Fighting ris...","[0.39222614840000003, 0.2332155477, 0.24028268...",train
15224,2020.0,61,Political system: Having the army rule,age,"[Fairly good, Very bad, Very good, Fairly bad]","[0.1740890688, 0.5020242915, 0.0971659919, 0.2...",train
15225,2022.0,Seperated,Important child qualities: hard work,married,"[mentioned, not mentioned]","[0.2666666667, 0.7333333333000001]",train
15226,2022.0,female,Confidence: The Police,gender,"[None at all, Not Very Much, Quite a lot, A gr...","[0.0587144623, 0.281829419, 0.5519159456, 0.10...",validation


### Add choice columns to dataset

In [ ]:
df=df_filtered_undersampled.progress_apply(choices,axis=1)

  0%|          | 0/15228 [00:00<?, ?it/s]

In [ ]:
df

,year,demographic,input,demographic_category,options,labels,split,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5
0,2020.0,17,"Member: Belong to conservation, the environmen...",age,"[not mentioned, mentioned]","[0.6857142857, 0.3142857143, -1.0, -1.0, -1.0,...",validation,not mentioned,mentioned,,,,
1,2021.0,48,Confidence: Armed Forces,age,"[A great deal , Quite a lot, Not Very Much, No...","[0.32, 0.32, 0.25, 0.11, -1.0, -1.0]",train,A great deal,Quite a lot,Not Very Much,None at all,,
2,2022.0,48,Justifiable: Death penalty,age,"[Always justifiable, Never justifiable]","[0.27118644070000003, 0.7288135593, -1.0, -1.0...",train,Always justifiable,Never justifiable,,,,
3,2020.0,Single,Protecting environment vs. Economic growth,married,[Protecting the environment should be given pr...,"[0.6173297966, 0.35349248450000004, 0.02917771...",train,Protecting the environment should be given pri...,Economic growth and creating jobs should be th...,Other answer,,,
4,2021.0,32,Satisfaction with your life,age,"[completely satisfied, Don't know]","[0.7206703911, 0.2793296089, -1.0, -1.0, -1.0,...",train,completely satisfied,Don't know,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15223,2020.0,62,Aims of respondent: first choice,age,"[maintaining order in the nation, Giving peopl...","[0.39222614840000003, 0.2402826855, 0.23321554...",train,maintaining order in the nation,Giving people more say in important government...,Fighting rising prices,Protecting freedom of speech,,
15224,2020.0,61,Political system: Having the army rule,age,"[Very good, Fairly good, Fairly bad, Very bad]","[0.0971659919, 0.1740890688, 0.2267206478, 0.5...",train,Very good,Fairly good,Fairly bad,Very bad,,
15225,2022.0,Seperated,Important child qualities: hard work,married,"[not mentioned, mentioned]","[0.7333333333000001, 0.2666666667, -1.0, -1.0,...",train,not mentioned,mentioned,,,,
15226,2022.0,female,Confidence: The Police,gender,"[A great deal , Quite a lot, Not Very Much, No...","[0.1075401731, 0.5519159456, 0.281829419, 0.05...",validation,A great deal,Quite a lot,Not Very Much,None at all,,


In [ ]:
if mode=='binary':
    df=df[df.labels<num_choices]
else:
    df=df[df.labels.map(len)==num_choices]
len(df)

15228

### Apply Templates

#### Zeroshot Setting for Masked Language Task


In [ ]:
df_filtered_undersampled

,year,demographic,input,demographic_category,options,labels,split
0,2020.0,17,"Member: Belong to conservation, the environmen...",age,"[not mentioned, mentioned]","[0.6857142857, 0.3142857143]",validation
1,2021.0,48,Confidence: Armed Forces,age,"[A great deal , Quite a lot, Not Very Much, No...","[0.32, 0.32, 0.25, 0.11]",train
2,2022.0,48,Justifiable: Death penalty,age,"[Always justifiable, Never justifiable]","[0.27118644070000003, 0.7288135593]",train
3,2020.0,Single,Protecting environment vs. Economic growth,married,[Protecting the environment should be given pr...,"[0.6173297966, 0.35349248450000004, 0.02917771...",train
4,2021.0,32,Satisfaction with your life,age,"[completely satisfied, Don't know]","[0.7206703911, 0.2793296089]",train
...,...,...,...,...,...,...,...
15223,2020.0,62,Aims of respondent: first choice,age,"[maintaining order in the nation, Giving peopl...","[0.39222614840000003, 0.2402826855, 0.23321554...",train
15224,2020.0,61,Political system: Having the army rule,age,"[Very good, Fairly good, Fairly bad, Very bad]","[0.0971659919, 0.1740890688, 0.2267206478, 0.5...",train
15225,2022.0,Seperated,Important child qualities: hard work,married,"[not mentioned, mentioned]","[0.7333333333000001, 0.2666666667]",train
15226,2022.0,female,Confidence: The Police,gender,"[A great deal , Quite a lot, Not Very Much, No...","[0.1075401731, 0.5519159456, 0.281829419, 0.05...",validation


In [ ]:
year_index = 0
demographic_index= 1
input_index = 2
category_index = 3
options_index = 4
labels_index = 5
split_index = 6


dataset_wvs_sentences = []
for input in df_filtered_undersampled.values:
    year = input[year_index]
    demographic = input[demographic_index]
    question_input = str(input[input_index])
    category = input[category_index]
    options = input[options_index]
    labels = input[labels_index]
    split = input[split_index]

    for i in range(len(options)):
        label = labels[i]
        choice = options[i]
        sentence = f"In {int(year)}, <mask> percent of {demographic} voted '{choice}' on the subject '{question_input}'".lower()
        dataset_wvs_sentences.append({"year":year,"demographic":demographic,"subject":question_input,"demographic_category":category,"choice":choice,"input": sentence, "y": label,"split":split})
dataset_wvs_sentences = pd.DataFrame(dataset_wvs_sentences)

In [ ]:
df[df["split"] == "validation"]

,year,demographic,input,demographic_category,options,labels,split,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5
0,2020.0,17,"Member: Belong to conservation, the environmen...",age,"[not mentioned, mentioned]","[0.6857142857, 0.3142857143, -1.0, -1.0, -1.0,...",validation,not mentioned,mentioned,,,,
9,2021.0,Kenya,Future changes: less importance placed on work,country,"[good, don't mind , bad]","[0.36854838710000004, 0.18548387100000002, 0.4...",validation,good,don't mind,bad,,,
13,2020.0,76,How often in country's elections: TV news favo...,age,"[Very Often, Fairly Often, Not often, Not at a...","[0.0873786408, 0.39805825240000003, 0.33009708...",validation,Very Often,Fairly Often,Not often,Not at all often,,
18,2021.0,60,Mother immigrant / born in country\n,age,"[No, Yes]","[0.1326530612, 0.8673469388, -1.0, -1.0, -1.0,...",validation,No,Yes,,,,
20,2021.0,"No, no paid employment: Retired/pensioned",Democracy: People receive state aid for unempl...,employement,"[not an essential characteristic of democracy,...","[0.20626631850000002, 0.7845953003, 0.00913838...",validation,not an essential characteristic of democracy,An essential characteristic of democracy,It is against democracy,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15214,2021.0,67,Aims of country: first choice,age,"[A high level of economic growth, Making sure ...","[0.4925373134, 0.2537313433, 0.1343283582, 0.1...",validation,A high level of economic growth,Making sure this country has strong defense fo...,Seeing that people have more say about how thi...,Trying to make our cities and countryside more...,,
15216,2021.0,45,Confidence: The Police,age,"[A great deal , Quite a lot, Not Very Much, No...","[0.09375000000000001, 0.3515625, 0.3203125, 0....",validation,A great deal,Quite a lot,Not Very Much,None at all,,
15218,2020.0,50,Confidence: The Police,age,"[A great deal , Quite a lot, Not Very Much, No...","[0.230958231, 0.4250614251, 0.2186732187, 0.12...",validation,A great deal,Quite a lot,Not Very Much,None at all,,
15222,2022.0,49,Duty towards society to have children,age,"[Strongly agree, Agree, Neither agree nor disa...","[0.156626506, 0.18072289160000002, 0.096385542...",validation,Strongly agree,Agree,Neither agree nor disagree,Disagree,Strongly Disagree,


In [ ]:
dataset_wvs_sentences['labels']=dataset_wvs_sentences['y']
try:
    del dataset_wvs_sentences['options']
except:
    pass
dataset_wvs_zeroshot=DatasetDict({k:datasets.Dataset.from_pandas(dataset_wvs_sentences[dataset_wvs_sentences['split']==k]) for k in {'train','test','validation'}})

### Zeroshot and fine-tuning setting for MCQ task

In [ ]:
df = df.progress_apply(apply_question_template,axis=1)

  0%|          | 0/15228 [00:00<?, ?it/s]

### Training, Validation and Test Sets

In [ ]:
df[df.labels.progress_map(lambda x: any(np.isnan(a) for a in x))]

  0%|          | 0/15228 [00:00<?, ?it/s]

,year,demographic,input,demographic_category,options,labels,split,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,question


In [ ]:
df['task'] = 0
df['input_ids'] = None
df['attention_mask'] = None

In [ ]:
try:
    del df['options']
except:
    pass
dataset=DatasetDict({k:datasets.Dataset.from_pandas(df[df['split']==k]) for k in {'train','test','validation'}})

In [ ]:
def show_one(example):
    print(f"Context: {example['question']}")
    print(f"  A - {example['question']} {example['choice_0']}")
    print(f"  B - {example['question']} {example['choice_1']}")
    print(f"  C - {example['question']} {example['choice_2']}")
    print(f"  D - {example['question']} {example['choice_3']}")
    print(f"  D - {example['question']} {example['choice_4']}")

In [ ]:
show_one(dataset["train"][8])

Context: 71: how much freedom of choice and control ?
  A - 71: how much freedom of choice and control ? a great deal of choice
  B - 71: how much freedom of choice and control ? no choice at all
  C - 71: how much freedom of choice and control ? 
  D - 71: how much freedom of choice and control ? 
  D - 71: how much freedom of choice and control ? 


In [ ]:
reco = tn.MultipleChoice(
    dataset,
    s1="question",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks_wvs = [reco]

## PollBERT Model

### Forward Method Binary

In [106]:
from transformers.models.roberta.modeling_roberta import *
def forward_binary(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], MultipleChoiceModelOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,
            num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors. (See
            `input_ids` above)
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]
        # print(input_ids)
        # print(tokenizer.decode(input_ids[0][0]))
        flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
        flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
        flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
        flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
        flat_inputs_embeds = (
            inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
            if inputs_embeds is not None
            else None
        )

        outputs = self.roberta(
            flat_input_ids,
            position_ids=flat_position_ids,
            token_type_ids=flat_token_type_ids,
            attention_mask=flat_attention_mask,
            head_mask=head_mask,
            inputs_embeds=flat_inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        reshaped_logits = logits.view(-1, num_choices)
        reshaped_logits[labels == -1] = 0
        labels[labels == -1] = 0
        loss = None
        if labels is not None:
            # move labels to correct device to enable model parallelism
            labels = labels.to(reshaped_logits.device)
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(reshaped_logits, labels)

        if not return_dict:
            output = (reshaped_logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return MultipleChoiceModelOutput(
            loss=loss,
            logits=reshaped_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

### Forward method Probabilities

In [107]:
from transformers.models.roberta.modeling_roberta import *


def mse_loss(input, target, ignored_index=-1):
    mask = target == ignored_index
    out = (input[~mask]-target[~mask])**2
    return out.mean()**.5

        
def forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    token_type_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.Tensor], MultipleChoiceModelOutput]:
    r"""
    labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
        Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,
        num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors. (See
        `input_ids` above)
    """
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]
    flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
    flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
    flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
    flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
    flat_inputs_embeds = (
        inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
        if inputs_embeds is not None
        else None
    )

    outputs = self.roberta(
        flat_input_ids,
        position_ids=flat_position_ids,
        token_type_ids=flat_token_type_ids,
        attention_mask=flat_attention_mask,
        head_mask=head_mask,
        inputs_embeds=flat_inputs_embeds,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )
    pooled_output = outputs[1]
    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)
    reshaped_logits = logits.view(-1, num_choices)
    
    reshaped_logits[labels==-1]=-100
    m=nn.Softmax(dim=1)
    reshaped_logits=m(reshaped_logits)
    
    #eshaped_logits[labels==-1]=0
    labels[labels==-1]=0

    loss = None
    if labels is not None:
        loss_fct = self.loss
        loss = loss_fct(reshaped_logits, labels)
    if not return_dict:
        output = (reshaped_logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output
    
    return MultipleChoiceModelOutput(
        loss=loss,
        logits=reshaped_logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )
from types import MethodType 

In [ ]:
from transformers.models.roberta.modeling_roberta import *
from xpflow import Xp
torch.autograd.set_detect_anomaly(False)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
        
def forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    token_type_ids: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    head_mask: Optional[torch.FloatTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.Tensor], MultipleChoiceModelOutput]:
    r"""
    labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
        Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,
        num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors. (See
        `input_ids` above)
    """
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]

    flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
    flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
    flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
    flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
    flat_inputs_embeds = (
        inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
        if inputs_embeds is not None
        else None
    )

    outputs = self.roberta(
        flat_input_ids,
        position_ids=flat_position_ids,
        token_type_ids=flat_token_type_ids,
        attention_mask=flat_attention_mask,
        head_mask=head_mask,
        inputs_embeds=flat_inputs_embeds,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

    pooled_output = outputs[1]
    
    pooled_output = self.dropout(pooled_output)
    
    logits = self.classifier(pooled_output)
    #print(self.classifier.in_features,self.classifier.out_features,self.classifier.bias)
    reshaped_logits = logits.view(-1, num_choices)
    #print(outputs[1].size(),pooled_output.size(),logits.size(),reshaped_logits.size()) 
    reshaped_logits[labels==-1]=-100
    m=nn.Softmax(dim=1)
    reshaped_logits=m(reshaped_logits)

    labels[labels==-1]=0# If working with BCE apply 0 on labels
    loss = None
    if labels is not None:
        loss_fct = self.loss
        loss = loss_fct(reshaped_logits, labels)
    if not return_dict:
        output = (reshaped_logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output
    
    return MultipleChoiceModelOutput(
        loss=loss,
        logits=reshaped_logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )
from types import MethodType 

### Loss Functions and metrics

In [108]:
from statistics import mean
def compute_metrics_old(element):
    f_distance = lambda x,y: np.abs((x-y)).max(axis=0).mean()
    y_pred,y_true = element
    distance = f_distance(y_pred,y_true)
    return distance
    
def crop(element):
  y_pred,y_true = element
  mask = y_true == -1
  return (y_pred[~mask],y_true[~mask])


def compute_metrics(self,batch):
  meta = {"name": self.name, "size": len(batch.predictions), "index": self.index}
  distance = mean([compute_metrics_old(crop(element)) for element in zip(batch.predictions,batch.label_ids)])
  return {**meta, 'distance':distance}

def compute_metrics_bce(self, eval_pred=None):
    f_distance = lambda x,y: np.abs((x-y)).max(axis=1).mean()
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    distance = f_distance(predictions,labels)
    meta = {"name": self.name, "size": len(predictions), "index": self.index}
    return {**meta, 'distance':distance}

def mse_loss(input, target, ignored_index=-1):
    mask = target == ignored_index
    out = (input[~mask]-target[~mask])**2
    return out.mean()**.5

def bce_loss(input, target, ignored_index=-1):
  loss = nn.BCELoss()
  mask = target == ignored_index
  out = loss(input[~mask],target[~mask])
  return out

def mse_loss_binary(input, target):
    out = (input-target)**2
    return out.mean()**.5

def bce_loss_binary(input, target):
  loss = nn.BCEWithLogitsLoss()
  out = loss(input,target)
  return out

# Experiments

## All Demographics

### GRID SEARCH - FineTuning Probabilities

#### Reddit Demo on Roberta

In [ ]:
class args:
    model_name = "roberta-base"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 3e-5
    num_train_epochs=10
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps = 1
    

models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][0])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.train()

#### WVS wihtout Reddit

In [ ]:
class args:
    model_name = "roberta-base"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps = 100
    

models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][0])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



 ------------------------------ START 1e-05 & 3 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.100900,0.084520,,208,0,0.170294
1,0.079100,0.094514,,208,0,0.191019
2,0.073900,0.115245,,208,0,0.203698


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5898, training_loss=0.08463457068090481, metrics={'train_runtime': 5138.5809, 'train_samples_per_second': 9.185, 'train_steps_per_second': 1.148, 'total_flos': 3.724389567347098e+16, 'train_loss': 0.08463457068090481, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.12094242870807648,
 'eval_name': '',
 'eval_size': 208,
 'eval_index': 0,
 'eval_distance': 0.20369841158390045,
 'eval_runtime': 12.0254,
 'eval_samples_per_second': 17.297,
 'eval_steps_per_second': 2.162,
 'epoch': 3.0,
 'Accuracy': nan}

In [ ]:
torch.cuda.empty_cache()

#### WVS with Reddit finetuned (Full)

In [ ]:
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=10
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    # max_steps = 2


models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])

trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

In [ ]:
trainer.model.task_models_list[0].push_to_hub("Qubix/RoBERTa-WVSMCQ7")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-WVSMCQ7/commit/33c1374113124445e6168c49527f0b084f70ca26', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='33c1374113124445e6168c49527f0b084f70ca26', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
class args:
    model_name = "Qubix/RoBERTa-WVSMCQ7"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=4
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8


models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class args:
    model_name = "Qubix/RoBERTa-WVSMCQ7"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2991941273212433,
 'eval_name': '',
 'eval_size': 209,
 'eval_index': 0,
 'eval_distance': 0.34720391035079956,
 'eval_runtime': 11.6235,
 'eval_samples_per_second': 17.981,
 'eval_steps_per_second': 2.323,
 'Accuracy': nan}

In [ ]:
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8
    max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.3640327751636505,
 'eval_name': '',
 'eval_size': 209,
 'eval_index': 0,
 'eval_distance': 0.5242656469345093,
 'eval_runtime': 14.946,
 'eval_samples_per_second': 13.984,
 'eval_steps_per_second': 1.807,
 'Accuracy': nan}

#### Fine-Tuning

In [ ]:
lr= 2e-5
bs = 8
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=1
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    max_steps=10


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][0])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()


 ------------------------------ START 2e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.721100,0.677038,,254,0,0.245166
1,0.686600,0.675966,,254,0,0.247967
2,0.685700,0.672981,,254,0,0.241259
3,0.683700,0.678294,,254,0,0.237856
4,0.680400,0.669603,,254,0,0.241177
5,0.677400,0.678003,,254,0,0.241691
6,0.672100,0.695510,,254,0,0.247369
8,0.669800,0.684228,,254,0,0.241389
8,0.667500,0.709295,,254,0,0.240832
9,0.663500,0.669670,,254,0,0.241026


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a

TrainOutput(global_step=13790, training_loss=0.6807825123079451, metrics={'train_runtime': 14026.0803, 'train_samples_per_second': 7.869, 'train_steps_per_second': 0.983, 'total_flos': 8.707923386523648e+16, 'train_loss': 0.6807825123079451, 'epoch': 10.0})

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()


 ------------------------------ START 3e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.728200,0.676117,,254,0,0.247778
1,0.688300,0.678519,,254,0,0.244788
2,0.690200,0.674162,,254,0,0.244503
3,0.683600,0.685420,,254,0,0.242411


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5516, training_loss=0.6975845215197833, metrics={'train_runtime': 5620.0712, 'train_samples_per_second': 7.855, 'train_steps_per_second': 0.981, 'total_flos': 3.483169354609459e+16, 'train_loss': 0.6975845215197833, 'epoch': 4.0})

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()


 ------------------------------ START 3e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.255700,0.217722,,254,0,0.251661
1,0.224500,0.260339,,254,0,0.244517
2,0.217600,0.200251,,254,0,0.241439
3,0.215000,0.158986,,254,0,0.238043


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5516, training_loss=0.22821001214683703, metrics={'train_runtime': 5592.4079, 'train_samples_per_second': 7.894, 'train_steps_per_second': 0.986, 'total_flos': 3.483169354609459e+16, 'train_loss': 0.22821001214683703, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub("Qubix/RoBERTa-PollBERT350K_full_model_BCE")
trainer.model.task_models_list[0].push_to_hub("Qubix/RoBERTa-PollBERT350K_BCE")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/7.49k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-PollBERT350K_BCE/commit/6e0a18a1d1e35d2ef2a770be57b5ed9a8c7fbb3e', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='6e0a18a1d1e35d2ef2a770be57b5ed9a8c7fbb3e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.per_device_train_batch_size)+" ---------------------------- \n ")
trainer.train()


 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.235000,0.216306,,254,0,0.242761
1,0.216700,0.255326,,254,0,0.240703
2,0.216300,0.200082,,254,0,0.239873
3,0.214600,0.165904,,254,0,0.239016
4,0.212700,0.236502,,254,0,0.241479
5,0.210200,0.207509,,254,0,0.238653
6,0.208000,0.251372,,254,0,0.239867
8,0.206800,0.216043,,254,0,0.242528
8,0.204900,0.245425,,254,0,0.244471
9,0.204300,0.201877,,254,0,0.243279


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a

TrainOutput(global_step=13790, training_loss=0.21295307427752788, metrics={'train_runtime': 13916.0006, 'train_samples_per_second': 7.931, 'train_steps_per_second': 0.991, 'total_flos': 8.707923386523648e+16, 'train_loss': 0.21295307427752788, 'epoch': 10.0})

## FINAL FINETUNING

#### Reddit Demo on RoBERTa-DEMO-CE

##### BCE

In [ ]:
lr= 3e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-RedditPoll350k_CE"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    gradient_accumulation_steps = bs
    #max_steps=10


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][0])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.6770937442779541,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.23930566012859344,
 'eval_runtime': 18.5027,
 'eval_samples_per_second': 13.728,
 'eval_steps_per_second': 1.729,
 'Accuracy': nan}

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 3e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.690100,0.681126,,254,0,0.242015
1,0.684800,0.680669,,254,0,0.241012
2,0.685400,0.677098,,254,0,0.240475
3,0.684100,0.684234,,254,0,0.240375


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5516, training_loss=0.6860881581351409, metrics={'train_runtime': 5225.0002, 'train_samples_per_second': 8.449, 'train_steps_per_second': 1.056, 'total_flos': 3.483169354609459e+16, 'train_loss': 0.6860881581351409, 'epoch': 4.0})

##### RMSE

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
trainer.train()# CE pre-trained model


 ------------------------------ START 3e-05 & 8 ---------------------------- 
 


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.218200,0.199604,,254,0,0.239428
1,0.218100,0.217414,,254,0,0.239556
2,0.217200,0.195366,,254,0,0.239475
3,0.216800,0.206370,,254,0,0.239537


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5516, training_loss=0.2175690482888557, metrics={'train_runtime': 5205.7003, 'train_samples_per_second': 8.481, 'train_steps_per_second': 1.06, 'total_flos': 3.485695220711424e+16, 'train_loss': 0.2175690482888557, 'epoch': 4.0})

#### Reddit Demo on Sileod/PollBERT

In [ ]:
lr= 1e-6
bs = 8
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=5
    per_device_train_batch_size = 24
    gradient_accumulation_steps = 1
    #max_steps=10


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['torch.nn.BCELoss()','mse_loss'][0])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics_bce,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2506578266620636,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.24124854803085327,
 'eval_runtime': 20.3946,
 'eval_samples_per_second': 12.454,
 'eval_steps_per_second': 1.569,
 'Accuracy': nan}

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.22488118708133698,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.2412484884262085,
 'eval_runtime': 22.1795,
 'eval_samples_per_second': 11.452,
 'eval_steps_per_second': 1.443,
 'Accuracy': nan}

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
wandb.init(project="polls", entity="quentintilman")
trainer.train()#NOW
wandb.finish()

In [ ]:
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
trainer.train()


 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,0.229800,0.215172,,254,0,0.241965
1,0.216500,0.256757,,254,0,0.241493
2,0.215900,0.199235,,254,0,0.239596
3,0.214800,0.152169,,254,0,0.239241


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=5516, training_loss=0.21923098934137622, metrics={'train_runtime': 5109.9778, 'train_samples_per_second': 8.64, 'train_steps_per_second': 1.079, 'total_flos': 3.483169354609459e+16, 'train_loss': 0.21923098934137622, 'epoch': 4.0})

In [ ]:
model, adapter = trainer.model.factorize()
model.push_to_hub("Qubix/RoBERTa-DemoPollBERT_RMSE")#best_model until now

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-DemoPollBERT_RMSE/commit/2b238a9b67ceb7fd91958f319c3ee4c94087503e', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='2b238a9b67ceb7fd91958f319c3ee4c94087503e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-DemoPollBERT_RMSE"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=10
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True))

AttributeError: ignored

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2050856649875641,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.239240825176239,
 'eval_runtime': 24.7084,
 'eval_samples_per_second': 10.28,
 'eval_steps_per_second': 1.295,
 'Accuracy': nan}

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-PollBERT350K"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=10
    per_device_train_batch_size=bs
    per_device_eval_batch_size=bs
    gradient_accumulation_steps = bs
    #max_steps=128


models = tn.Model(tasks_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True))

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.20897020399570465,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.23723392188549042,
 'eval_runtime': 25.5036,
 'eval_samples_per_second': 9.959,
 'eval_steps_per_second': 1.255,
 'Accuracy': nan}

In [ ]:
tasks_reddit_demographics_plbrt

[MultipleChoice(dataset=DatasetDict({
     train: Dataset({
         features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'task', 'labels', '__index_level_0__'],
         num_rows: 11037
     })
     validation: Dataset({
         features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'task', 'labels', '__index_level_0__'],
         num_rows: 254
     })
     test: Dataset({
         features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'task', 'labels', '__index_level_0__'],
         num_rows: 265
     })
 }), name='', tokenizer=None, tokenizer_kwargs=frozendict.frozendict({'padding': 'max_length', 'max_length': 256, 'truncation': True}), max_rows=None, max_rows_eval=None, oversamplin

# LATEST

In [ ]:
from xpflow import Xp
lr = 1e-5
bs = 24
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=10
      #batch_size =bs
      logging_strategy = "steps"
      warmup_ratio=0.06
      loss = ['torch.nn.BCELoss()','mse_loss'][0]
      gradient_accumulation_steps = 8



reco = tn.MultipleChoice(
    dataset_reddit_demographics_plbrt,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks = [reco]

for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  models.task_models_list[0].loss = eval(args.loss)
  models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
  tasks[0].compute_metrics=MethodType(compute_metrics_bce,tasks[0])
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()
  wandb.finish()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
0,No log,0.271851,,30,0,0.191888
1,No log,0.231168,,30,0,0.191640
2,No log,0.277343,,30,0,0.191470
4,0.244200,0.253853,,30,0,0.191469
4,0.244200,0.231521,,30,0,0.191848
5,0.244200,0.230687,,30,0,0.190770
6,0.243500,0.272897,,30,0,0.201315
8,0.243500,0.245394,,30,0,0.219391
8,0.243500,0.253714,,30,0,0.227071
9,0.243500,0.226708,,30,0,0.224246


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a

/epoch,▁▂▃▃▄▅▆▆▇█
/eval_distance,▁▁▁▁▁▁▃▇█▇
/eval_index,▁▁▁▁▁▁▁▁▁▁
/eval_loss,▇▂█▅▂▂▇▄▅▁
/eval_runtime,█▆▅▄█▄▃▆▄▁
/eval_samples_per_second,▁▂▂▄▁▄▄▂▄█
/eval_size,▁▁▁▁▁▁▁▁▁▁
/eval_steps_per_second,▁▂▂▄▁▄▄▂▄█
epoch,▁▅█
gradient_accumulation_steps,▁
learning_rate,█▅▁


In [ ]:
# model, adapter = trainer.model.factorize()
# model.push_to_hub("Qubix/RoBERTa-DemoPollBERT_BCE_latest")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-DemoPollBERT_BCE_latest/commit/629a345a52dc464a4113c384bde646ef2b4e42b8', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='629a345a52dc464a4113c384bde646ef2b4e42b8', pr_url=None, pr_revision=None, pr_num=None)

### WVS with Reddit finetuned (Full + Demographics)

In [ ]:
from xpflow import Xp
lr = 1e-5
bs = 24
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=10
      #batch_size =bs
      logging_strategy = "steps"
      warmup_ratio=0.06
      loss = ['torch.nn.BCELoss()','mse_loss'][0]
      gradient_accumulation_steps = 8



reco = tn.MultipleChoice(
    dataset_reddit_demographics_plbrt,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks = [reco]

for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  models.task_models_list[0].loss = eval(args.loss)
  models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
  tasks[0].compute_metrics=MethodType(compute_metrics_bce,tasks[0])
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()
  wandb.finish()

In [ ]:
class args:
    model_name = "Qubix/demoPollBERT"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 1e-5
    num_train_epochs=3
    per_device_train_batch_size=8
    per_device_eval_batch_size=8
    gradient_accumulation_steps = 8

models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['torch.nn.BCELoss()','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

In [ ]:
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default
print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.num_train_epochs)+" ---------------------------- \n ")
trainer.train()

In [ ]:
model, adapter = trainer.model.factorize()
model.push_to_hub("Qubix/RoBERTa-WVMCQ7-BCE")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-WVMCQ7-BCE/commit/1fc1c654f7076f3ae9aa0c2b95f415181a59a148', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='1fc1c654f7076f3ae9aa0c2b95f415181a59a148', pr_url=None, pr_revision=None, pr_num=None)

### RoBERTa-WVMCQ7

In [ ]:
from xpflow import Xp
lr = 1e-3
bs = 24
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=10
      #batch_size =bs
      logging_strategy = "steps"
      warmup_ratio=0.06
      loss = ['torch.nn.BCELoss()','mse_loss'][0]
      gradient_accumulation_steps = 8



reco = tn.MultipleChoice(
    dataset,
    s1="question",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks = [reco]

for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  models.task_models_list[0].loss = eval(args.loss)
  models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
  tasks[0].compute_metrics=MethodType(compute_metrics_bce,tasks[0])
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()
  wandb.finish()

cmd_args: {'f': '/root/.local/share/jupyter/runtime/kernel-1f83b884-1637-4a2f-847b-d45ba29fa4f0.json'}


wandb: ERROR Control-C detected -- Run data was not synced


Problem at: <ipython-input-142-0abca1401224> 29 <cell line: 28>


CommError: ignored

# Evaluation

## Zeroshot

### Loading model

In [ ]:
classifier = transformers.pipeline("fill-mask",device=0,model="roberta-base")

In [ ]:
def softmax(x):
   return np.exp(x)/sum(np.exp(x))

In [ ]:
from decimal import *
def perform_zeroshot(x):
  title = x["title"]
  year = x["created"]
  label = x["labels"]
  question_input = x["title"]
  choices = [x[f'choice_{i}'] for i in range((sum([1 for l in label if l != -1])))]
  zeroshot_preds = []
  try:
    for choice in choices:
      zeroshot_preds.append(round(Decimal(int(classifier(f"In {year}, <mask> percent of people voted '{choice}' on the question '{question_input}'".lower())[0]["token_str"]) / 100),2))
    x["zeroshot_preds"] = softmax(zeroshot_preds)
  except:
    pass
  return x

In [ ]:
from decimal import *
def perform_zeroshot_demographic(x):
  title = x["title"]
  year = x["created"]
  label = x["labels"]
  demographic = x["demographic"]
  question_input = x["title"]
  choices = [x[f'choice_{i}'] for i in range((sum([1 for l in label if l != -1])))]
  zeroshot_preds = []
  try:
    for choice in choices:
      zeroshot_preds.append(round((int(classifier(f"In {year}, <mask> percent of {demographic} voted '{choice}' on the question '{question_input}'".lower())[0]["token_str"]) / 100),2))
    x["zeroshot_preds"] = np.array(softmax(zeroshot_preds))
  except:
    pass
  return x

In [ ]:
from decimal import *
def perform_zeroshot_demographic_wvs(x):
  year = x["year"]
  label = x["labels"]
  demographic = x["demographic"]
  question_input = x["input"]
  choices = [x[f'choice_{i}'] for i in range((sum([1 for l in label if l != -1])))]
  zeroshot_preds = []
  try:
    for choice in choices:
      zeroshot_preds.append(round((int(classifier(f"In {year}, <mask> percent of {demographic} voted '{choice}' on the subject '{question_input}'".lower())[0]["token_str"]) / 100),2))
    x["zeroshot_preds"] = np.array(softmax(zeroshot_preds))
  except:
    pass
  return x

### Prediction

#### WVMCQ7

In [ ]:
df_wvs_zeroshot = pd.DataFrame(tasks_wvs[0].dataset["validation"])

In [ ]:
df_wvs_zeroshot = df_wvs_zeroshot.progress_apply(perform_zeroshot_demographic_wvs,axis=1)

  0%|          | 0/3154 [00:00<?, ?it/s]

#### RedditPoll350k (Full Dataset)

In [ ]:
df_reddit_zeroshot = df_reddit_zeroshot.progress_apply(to_year,axis=1)

  0%|          | 0/7863 [00:00<?, ?it/s]

In [ ]:
df_reddit_zeroshot = df_reddit_zeroshot.progress_apply(perform_zeroshot,axis=1)

  0%|          | 0/7863 [00:00<?, ?it/s]

In [ ]:
df_reddit_zeroshot = df_reddit_zeroshot.dropna()
df_reddit_zeroshot.reset_index(inplace=True)

In [ ]:
df_reddit_zeroshot.to_json("roberta_zeroshot_results_RedditPoll350k.json")

#### DEMO NNL

In [ ]:
demo_nnl = df_reddit_demographics_normalized[df_reddit_demographics_normalized["split"] == "validation"]
df_demo_reddit_zeroshot_nnl = pd.DataFrame(demo_nnl).progress_apply(perform_zeroshot_demographic,axis=1)

  0%|          | 0/254 [00:00<?, ?it/s]

In [ ]:
df_demo_reddit_zeroshot_nnl[df_demo_reddit_zeroshot_nnl["title"] == "Is Africa a country?"]

,id,title,created,category,inputs,n_answers,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task,zeroshot_preds,rmse
71769,v3qyw3,Is Africa a country?,1654229271,news_&_social_concern,Is Africa a country?,131,validation,American,Yes,No,IGN,IGN,IGN,IGN,"[0.04819277108433735, 0.9518072289156626, -1.0...",0,"[0.5, 0.5]","tensor(0.4518, dtype=torch.float64)"
71769,v3qyw3,Is Africa a country?,1654229271,news_&_social_concern,Is Africa a country?,131,validation,Non-American,Yes,No,IGN,IGN,IGN,IGN,"[0.020833333333333332, 0.9791666666666666, -1....",0,"[0.5, 0.5]","tensor(0.4792, dtype=torch.float64)"
194651,p4cx41,Is Africa a country?,1628964228,news_&_social_concern,Is Africa a country?,163,validation,American,Yes,No,IGN,IGN,IGN,IGN,"[0.13924050632911392, 0.8607594936708861, -1.0...",0,"[0.5, 0.5]","tensor(0.3608, dtype=torch.float64)"
194651,p4cx41,Is Africa a country?,1628964228,news_&_social_concern,Is Africa a country?,163,validation,Not American,Yes,No,IGN,IGN,IGN,IGN,"[0.09090909090909091, 0.9090909090909091, -1.0...",0,"[0.5, 0.5]","tensor(0.4091, dtype=torch.float64)"


#### RedditPoll350k (Demographic Subset)

In [ ]:
df_demo_reddit_zeroshot = pd.DataFrame(df_demo_reddit_zeroshot).progress_apply(perform_zeroshot_demographic,axis=1)

  0%|          | 0/254 [00:00<?, ?it/s]

In [ ]:
df_demo_reddit_zeroshot.to_json("roberta_zeroshot_results_DemoRedditPoll350k.json")

## Constant Predictor

In [ ]:
def constant_prediction(x):
    try:
      x["constant_preds"] = [1/len(x['zeroshot_preds'])]*len(x["labels"])
      return x
    except:
      pass

In [ ]:
constant_results_reddit_bert = df_reddit_zeroshot.progress_apply(constant_prediction,axis=1)
#constant_prediction_demo_reddit = results_reddit_bert.progress_apply(constant_prediction,axis=1)

  0%|          | 0/7954 [00:00<?, ?it/s]

In [ ]:
constant_results_wvs = df_wvs_zeroshot.progress_apply(constant_prediction,axis=1)

  0%|          | 0/3154 [00:00<?, ?it/s]

# Results

## Zeroshot RoBERTa

#### Metrics

In [ ]:
def rmse(x):
  try:
    preds = torch.tensor([float(v) for v in x["zeroshot_preds"]])
    labels = [ l for l in x["labels"] if l != -1]
    x["rmse"] = mse_loss(preds,torch.tensor(labels))
    return x
  except:
      pass

#### RedditPoll350K (Full)

In [ ]:
results_reddit_bert = pd.read_json("./roberta_zeroshot_results_RedditPoll350k.json")

In [ ]:
results_reddit_bert = df_reddit_zeroshot.progress_apply(rmse,axis=1)

  0%|          | 0/7855 [00:00<?, ?it/s]

In [ ]:
np.mean([t.item() for t in [v for v in results_reddit_bert["rmse"].values if v != None]])

0.17648083984805596

#### RedditPoll350k (not normalized demo subset)

In [ ]:
df_demo_reddit_zeroshot = df_demo_reddit_zeroshot.progress_apply(rmse,axis=1)

  0%|          | 0/254 [00:00<?, ?it/s]

In [ ]:
df_demo_reddit_zeroshot_nnl

,id,title,created,category,inputs,...,choice_5,label,task,zeroshot_preds,rmse
557,ysxe6t,When’s the last time you saw a medical doctor?,1668230491,fitness_&_health,When’s the last time you saw a medical doctor?,...,IGN,"[0.6039603960396039, 0.12871287128712872, 0.26...",0,"[0.3333333333333333, 0.3333333333333333, 0.333...","tensor(0.1996, dtype=torch.float64)"
2056,yp3xnr,Have you finished a book this year?,1667862369,arts_&_culture,Have you finished a book this year?,...,IGN,"[0.5738636363636364, 0.42613636363636365, -1.0...",0,"[0.574442516811659, 0.42555748318834097]","tensor(0.0006, dtype=torch.float64)"
2056,yp3xnr,Have you finished a book this year?,1667862369,arts_&_culture,Have you finished a book this year?,...,IGN,"[0.7023809523809523, 0.2976190476190476, -1.0,...",0,"[0.574442516811659, 0.42555748318834097]","tensor(0.1279, dtype=torch.float64)"
2554,yo07in,Should the requirement to drink and gamble be ...,1667763280,diaries_&_daily_life,Should the requirement to drink and gamble be ...,...,IGN,"[0.29, 0.71, -1.0, -1.0, -1.0, -1.0]",0,"[0.5, 0.5]","tensor(0.2100, dtype=torch.float64)"
2554,yo07in,Should the requirement to drink and gamble be ...,1667763280,diaries_&_daily_life,Should the requirement to drink and gamble be ...,...,IGN,"[0.26436781609195403, 0.735632183908046, -1.0,...",0,"[0.5374298453437496, 0.46257015465625045]","tensor(0.2731, dtype=torch.float64)"
...,...,...,...,...,...,...,...,...,...,...,...
298246,i3ar6l,Would you be in a relationship with a sentient...,1596507080,relationships,Would you be in a relationship with a sentient...,...,IGN,"[0.25396825396825395, 0.746031746031746, -1.0,...",0,"[0.574442516811659, 0.42555748318834097]","tensor(0.3205, dtype=torch.float64)"
298246,i3ar6l,Would you be in a relationship with a sentient...,1596507080,relationships,Would you be in a relationship with a sentient...,...,IGN,"[0.3333333333333333, 0.6666666666666666, -1.0,...",0,"[0.5, 0.5]","tensor(0.1667, dtype=torch.float64)"
300276,hwc4z7,Which is more attractive to you; a guy with fe...,1595493430,celebrity_&_pop_culture,Which is more attractive to you; a guy with fe...,...,IGN,"[0.46153846153846156, 0.5384615384615384, -1.0...",0,"[0.5, 0.5]","tensor(0.0385, dtype=torch.float64)"
300276,hwc4z7,Which is more attractive to you; a guy with fe...,1595493430,celebrity_&_pop_culture,Which is more attractive to you; a guy with fe...,...,IGN,"[0.6923076923076923, 0.3076923076923077, -1.0,...",0,"[0.5, 0.5]","tensor(0.1923, dtype=torch.float64)"


In [ ]:
np.mean([t.item() for t in df_demo_reddit_zeroshot["rmse"].values])

0.22311911528248304

#### RedditPoll350K (Demographic Subset)

In [ ]:
df_demo_reddit_zeroshot_nnl = df_demo_reddit_zeroshot_nnl.progress_apply(rmse,axis=1)

  0%|          | 0/254 [00:00<?, ?it/s]

In [ ]:
results_demo_reddit_bert = pd.read_json("./roberta_zeroshot_results_DemoRedditPoll350k.json")

#### WVMCQ7

In [ ]:
df_wvs_zeroshot = df_wvs_zeroshot.progress_apply(rmse,axis=1)

  0%|          | 0/3154 [00:00<?, ?it/s]

In [ ]:
np.mean([t.item() for t in [v for v in df_wvs_zeroshot["rmse"].values if v != None]])

0.20790720585131484

## Constant Predition Model

In [ ]:
def rmse_constant(x):
  try:
    x["rmse"] = mse_loss(torch.tensor(x["constant_preds"]),torch.tensor(x["labels"]))
    return x
  except:
    pass

#### RedditPoll350k (Full)

In [ ]:
results_reddit_bert = results_reddit_bert.progress_apply(rmse_constant,axis=1)
np.mean([t.item() for t in results_reddit_bert["rmse"].values])

#### RedditPoll350k (demographic subset)


In [ ]:
constant_prediction_demo_reddit = constant_prediction_demo_reddit.progress_apply(rmse_constant,axis=1)
np.mean([t.item() for t in constant_prediction_demo_reddit["rmse"].values])

  0%|          | 0/254 [00:00<?, ?it/s]

0.22007513739989967

#### WVMCQ7

In [ ]:
constant_results_wvs = constant_results_wvs.progress_apply(rmse_constant,axis=1)
constant_results_wvs = constant_results_wvs.dropna()
np.mean([t.item() for t in constant_results_wvs["rmse"].values])

  0%|          | 0/3154 [00:00<?, ?it/s]

0.2035907033068863

## Fine-tuned Model

### RedditPoll350k

#### RedditPoll350k (Full)

In [ ]:
lr= 1e-5
bs = 1
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    gradient_accumulation_steps = bs
    per_device_train_batch_size=24
    warmup_ratio=0.06
    #max_steps=10


models = tn.Model(tasks_reddit_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.18192410469055176,
 'eval_name': '',
 'eval_size': 7863,
 'eval_index': 0,
 'eval_distance': 0.24564938247203827,
 'eval_runtime': 488.4877,
 'eval_samples_per_second': 16.097,
 'eval_steps_per_second': 2.012,
 'Accuracy': nan}

#### RedditPoll350k (demographic subset)


In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-DemoPollBERT_RMSE"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    gradient_accumulation_steps = bs
    #max_steps=10


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.20392562448978424,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.23556974530220032,
 'eval_runtime': 16.4586,
 'eval_samples_per_second': 15.433,
 'eval_steps_per_second': 1.944,
 'Accuracy': nan}

#### WVMCQ7

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "sileod/plbrt"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    gradient_accumulation_steps = bs
    #max_steps=10


models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.20433157682418823,
 'eval_name': '',
 'eval_size': 3154,
 'eval_index': 0,
 'eval_distance': 0.24242714047431946,
 'eval_runtime': 270.7916,
 'eval_samples_per_second': 11.647,
 'eval_steps_per_second': 1.459,
 'Accuracy': nan}

### RedditPoll350K (finetuned on demographic subset)

#### Evaluated RedditPoll350k (Full)

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-DemoPollBERT_RMSE"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    gradient_accumulation_steps = bs
    #max_steps=10


models = tn.Model(tasks_reddit_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.17928487062454224,
 'eval_name': '',
 'eval_size': 7863,
 'eval_index': 0,
 'eval_distance': 0.2417851835489273,
 'eval_runtime': 564.1689,
 'eval_samples_per_second': 13.937,
 'eval_steps_per_second': 1.742,
 'Accuracy': nan}

#### RedditPoll350k (demographic subset)


In [ ]:

lr= 1e-5
bs = 8
class args:
    model_name = "Qubix/RoBERTa-DemoPollBERT_RMSE"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    gradient_accumulation_steps = bs
    #max_steps=10


models = tn.Model(tasks_reddit_demographics_plbrt, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_reddit_demographics_plbrt[0].compute_metrics=MethodType(compute_metrics,tasks_reddit_demographics_plbrt[0])
trainer = tn.Trainer(models, tasks_reddit_demographics_plbrt, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.20392562448978424,
 'eval_name': '',
 'eval_size': 254,
 'eval_index': 0,
 'eval_distance': 0.23556974530220032,
 'eval_runtime': 20.3099,
 'eval_samples_per_second': 12.506,
 'eval_steps_per_second': 1.576,
 'Accuracy': nan}

#### WVMCQ7

In [ ]:
lr= 1e-5
bs = 8
class args:
    model_name ="Qubix/RoBERTa-DemoPollBERT_RMSE"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = lr
    num_train_epochs=4
    gradient_accumulation_steps = bs
    #max_steps=10


models = tn.Model(tasks_wvs, args,) # list of models; by default, shared encoder, task-specific CLS token task-specific head 

models.task_models_list[0].loss = eval(['bce_loss','mse_loss'][1])
models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
tasks_wvs[0].compute_metrics=MethodType(compute_metrics,tasks_wvs[0])
trainer = tn.Trainer(models, tasks_wvs, args, tokenizer = AutoTokenizer.from_pretrained("roberta-large", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.19929759204387665,
 'eval_name': '',
 'eval_size': 3154,
 'eval_index': 0,
 'eval_distance': 0.23957796394824982,
 'eval_runtime': 273.3901,
 'eval_samples_per_second': 11.537,
 'eval_steps_per_second': 1.445,
 'Accuracy': nan}

## P-value significant Question Subset

In [ ]:
df_reddit_sgn = pd.DataFrame([item  for item in df_reddit_demographics_normalized_labels.values if item[1] in redditPoll350k_significant_reduced["question"].values],columns = df_reddit_demographics_normalized_labels.columns)
df_reddit_sgn['input_ids'] = 0
df_reddit_sgn['attention_mask'] = 0
df_reddit_sgn =  pd.DataFrame([item for item in df_reddit_sgn.values if df_reddit_sgn["demographic"].value_counts()[item[6]]>49],columns=df_reddit_sgn.columns)
df_reddit_sgn = df_reddit_sgn.undersample(column="demographic")
df_reddit_sgn = df_reddit_sgn.progress_apply(to_not,axis=1)
df_reddit_sgn["inputs"] = df_reddit_sgn["demographic"] +": "+df_reddit_sgn["title"]
df_reddit_sgn['split']=df_reddit_sgn.title.map(lambda x: get_split(x,0.90,0.10))
dataset_reddit_sgn=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_sgn[df_reddit_sgn['split']==k]) for k in {'train','test','validation'}})

  0%|          | 0/692 [00:00<?, ?it/s]

In [ ]:
dataset_reddit_sgn['validation'].to_json("validation_set_sgn_reddit.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

35437

In [ ]:
from xpflow import Xp
lr = 5e-5
bs = 24
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=20
      #batch_size =bs
      logging_strategy = "steps"
      warmup_ratio=0.06
      loss = ['torch.nn.BCELoss()','mse_loss'][0]
      gradient_accumulation_steps = 8



reco = tn.MultipleChoice(
    dataset_reddit_sgn,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks = [reco]

for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  models.task_models_list[0].loss = eval(args.loss)
  models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
  tasks[0].compute_metrics=MethodType(compute_metrics_bce,tasks[0])
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
  print(trainer.evaluate())
  print(trainer.train())
  wandb.finish()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.27201151847839355, 'eval_name': '', 'eval_size': 68, 'eval_index': 0, 'eval_distance': 0.2376537322998047, 'eval_runtime': 6.0837, 'eval_samples_per_second': 11.177, 'eval_steps_per_second': 1.479, 'Accuracy': nan}


Epoch,Training Loss,Validation Loss,Name,Size,Index,Distance
1,No log,0.241312,,68,0,0.237645
2,No log,0.246469,,68,0,0.237828
3,No log,0.253649,,68,0,0.240216
4,No log,0.243050,,68,0,0.241405
5,No log,0.246866,,68,0,0.239226
6,No log,0.243307,,68,0,0.238445
7,0.238000,0.283069,,68,0,0.236058
8,0.238000,0.225285,,68,0,0.235810
9,0.238000,0.248196,,68,0,0.229535
10,0.238000,0.260210,,68,0,0.230050


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a

TrainOutput(global_step=780, training_loss=0.23772437266814403, metrics={'train_runtime': 770.0516, 'train_samples_per_second': 8.103, 'train_steps_per_second': 1.013, 'total_flos': 4925438898831360.0, 'train_loss': 0.23772437266814403, 'epoch': 10.0})


/epoch,▁▂▃▃▄▅▆▆▇█
/eval_distance,▆▆▇█▇▆▅▅▁▁
/eval_index,▁▁▁▁▁▁▁▁▁▁
/eval_loss,▃▄▄▃▄▃█▁▄▅
/eval_runtime,▇▆▁▃█▃▁▄▁▇
/eval_samples_per_second,▂▂█▅▁▅█▅█▂
/eval_size,▁▁▁▁▁▁▁▁▁▁
/eval_steps_per_second,▂▂█▅▁▅█▅█▂
epoch,▁█
gradient_accumulation_steps,▁
learning_rate,█▁


In [ ]:
model, adapter = trainer.model.factorize()
model.push_to_hub("Qubix/RoBERTa-RP350k-spv-regression-latest")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-RP350k-spv-regression-latest/commit/884dbd4bcff4452b2c0e80e8fbba9a022861729f', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='884dbd4bcff4452b2c0e80e8fbba9a022861729f', pr_url=None, pr_revision=None, pr_num=None)

# Classification Task

### Classification Set Up

In [131]:
def to_binary(x):
  x["labels"] = np.argmax(x["labels"])
  return x

def to_not(x):
  demo = x["demographic"]
  if demo == "non-american":
    demo = "not american"
  x["demographic"] = demo
  return x

In [ ]:
#df_reddit_demographics_normalized_labels = df_reddit_demographics_normalized_labels.progress_apply(to_not,axis=1)

  0%|          | 0/11556 [00:00<?, ?it/s]

In [132]:
df_reddit_demographics_normalized_labels["labels"] = df_reddit_demographics_normalized_labels["label"]

In [133]:
df_reddit_binary = df_reddit_demographics_normalized_labels.progress_apply(to_binary,axis=1)

  0%|          | 0/11556 [00:00<?, ?it/s]

In [134]:
df_reddit_binary_sign = pd.DataFrame([item  for item in df_reddit_binary.values if item[1] in redditPoll350k_significant_reduced["question"].values],columns = df_reddit_binary.columns)

In [135]:
df_reddit_binary_sign['input_ids'] = 0
df_reddit_binary_sign['attention_mask'] = 0

In [136]:
df_reddit_reduced =  pd.DataFrame([item for item in df_reddit_binary_sign.values if df_reddit_binary_sign["demographic"].value_counts()[item[6]]>49],columns=df_reddit_binary_sign.columns)
df_reddit_reduced_undersampled = df_reddit_reduced.undersample(column="demographic")

In [137]:
df_reddit_reduced_undersampled["inputs"] = df_reddit_reduced_undersampled["demographic"] +": "+df_reddit_reduced_undersampled["title"]

In [139]:
df_reddit_reduced['split']=df_reddit_reduced.title.map(lambda x: get_split(x))
dataset_reddit_binary_sign=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_reduced[df_reddit_reduced['split']==k]) for k in {'train','test','validation'}})
dataset_reddit_binary_sign

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 1145
    })
    validation: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 123
    })
    test: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 0
    })
})

In [141]:
df_reddit_reduced_undersampled['split']=df_reddit_reduced_undersampled.title.map(lambda x: get_split(x))
dataset_reddit_binary_sign=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_reduced_undersampled[df_reddit_reduced_undersampled['split']==k]) for k in {'train','test','validation'}})
dataset_reddit_binary_sign

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 310
    })
    validation: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 30
    })
    test: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 0
    })
})

## EasyPollBERT (Binarised Classification Task)

### RedditPoll350k demographics

#### FineTuning Binary

In [ ]:
# lr = 3e-5
# bs = 8
# class args:
#       model_name = "sileod/plbrt"
#       # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
#       learning_rate = lr
#       num_train_epochs=4
#       per_device_train_batch_size=bs
#       per_device_eval_batch_size=bs
#       #gradient_accumulation_steps = bs#dont need this as batch already makes batches of 8 examples
#       # max_steps=10

# reco = tn.MultipleChoice(
#     dataset_reddit_demographics_plbrt,
#     s1="inputs",
#     y="labels",
#     num_labels = num_choices,
#     choices=choices_names,
# )

# tasks = [reco]
# models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
# models.task_models_list[0].loss = torch.nn.CrossEntropyLoss()
# models.task_models_list[0].forward = MethodType(forward_binary, models.task_models_list[0])
# trainer = tn.Trainer(models, tasks, args, tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default

In [ ]:
# print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
# wandb.init(project="polls", entity="quentintilman")
# trainer.train()#NOW
# wandb.finish()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Name,Size,Index
1,0.991200,0.821015,0.429134,0.388414,,254,0
2,0.775300,0.782752,0.429134,0.346177,,254,0
3,0.773200,0.820998,0.488189,0.447020,,254,0
4,0.771600,0.728498,0.433071,0.358325,,254,0


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=44168, training_loss=0.8277981230755298, metrics={'train_runtime': 13882.7801, 'train_samples_per_second': 3.181, 'train_steps_per_second': 3.181, 'total_flos': 3.4970616181702656e+16, 'train_loss': 0.8277981230755298, 'epoch': 4.0})

### RedditPoll350k demographics (significant p-value subset)

In [144]:
from xpflow import Xp
lr = 1e-5
bs = 8
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=20
      batch_size =bs
      loss = ["torch.nn.CrossEntropyLoss()"]
      gradient_accumulation_steps = 8#dont need this as batch already makes batches of 8 examples

reco = tn.MultipleChoice(
    dataset_reddit_binary_sign,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
)
for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.batch_size)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()#NOW
  wandb.finish()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


  0%|          | 0/30 [00:00<?, ?it/s]

Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
c:\Python35\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/760 [00:00<?, ?it/s]

...

{'eval_loss': 1.6031155586242676, 'eval_accuracy': 0.7, 'eval_f1': 0.681962481962482, 'eval_name': '', 'eval_size': 30, 'eval_index': 0, 'eval_runtime': 2.3413, 'eval_samples_per_second': 12.813, 'eval_steps_per_second': 1.708, 'epoch': 18.99}
{'loss': 0.6086, 'learning_rate': 0.0, 'epoch': 19.61}


  0%|          | 0/30 [00:00<?, ?it/s]

Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.7345932722091675, 'eval_accuracy': 0.7, 'eval_f1': 0.681962481962482, 'eval_name': '', 'eval_size': 30, 'eval_index': 0, 'eval_runtime': 2.7229, 'eval_samples_per_second': 11.018, 'eval_steps_per_second': 1.469, 'epoch': 19.61}
{'train_runtime': 923.8851, 'train_samples_per_second': 6.711, 'train_steps_per_second': 0.823, 'train_loss': 0.7265229149868614, 'epoch': 19.61}


/epoch,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇██
/eval_accuracy,▃▆▃▃▃▃▃▃▅▃▁▅▆▇▇█████
/eval_f1,▄▃▄▅▄▄▄▅▆▄▁▃▇▇▇█████
/eval_index,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
/eval_loss,▃▃▂▃▄▂▃▂▂▃▂▂▁▅▂▃█▃▆▆
/eval_runtime,▃▅▂▂▃▂▂▄▃█▅▃▁▂▂▂▄▂▁▆
/eval_samples_per_second,▆▄▇▇▆▇▇▄▆▁▄▆█▇▇▆▅▇█▃
/eval_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
/eval_steps_per_second,▆▄▇▇▆▇▇▄▆▁▄▆█▇▇▆▅▇█▃
batch_size,▁
epoch,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇██


In [ ]:
from xpflow import Xp
lr = 1e-5
bs = 8
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=20
      batch_size =bs
      loss = ["torch.nn.CrossEntropyLoss()"]
      gradient_accumulation_steps = 1#dont need this as batch already makes batches of 8 examples

reco = tn.MultipleChoice(
    dataset_reddit_binary_sign,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
)
for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.batch_size)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()#NOW
  wandb.finish()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Name,Size,Index
1,0.946000,0.717496,0.512821,0.444444,,39,0
2,0.918400,0.815426,0.487179,0.380952,,39,0
3,0.911600,0.951533,0.512821,0.444444,,39,0
4,1.008700,0.513327,0.487179,0.306646,,39,0
5,1.364000,2.513869,0.512821,0.444444,,39,0
6,1.352400,3.425115,0.641026,0.610740,,39,0
7,1.525600,1.244719,0.692308,0.682540,,39,0
8,1.420100,2.324319,0.666667,0.663114,,39,0
9,1.525200,0.154998,0.641026,0.592262,,39,0
10,1.308700,4.219237,0.538462,0.491453,,39,0


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a

/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
/eval_accuracy,▂▁▂▁▂▆▇▆▆▃▆▆▆▄▇▆▅█▇▆
/eval_f1,▃▂▃▁▃▆█▇▆▄▇▇▆▆█▇▇██▇
/eval_index,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
/eval_loss,▂▂▂▂▅▆▃▄▁▇▄▄▁▆▁▅▇█▄▄
/eval_runtime,▅███▅▁▄▆▇▇▅▁▃▄▅▆▇▅▄▅
/eval_samples_per_second,▃▁▁▁▂█▄▂▁▁▃█▅▄▃▂▁▃▄▃
/eval_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
/eval_steps_per_second,▃▁▁▁▂█▄▂▁▁▃█▅▄▃▂▁▃▄▃
batch_size,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


In [ ]:
model, adapter = trainer.model.factorize()
model.push_to_hub("Qubix/RoBERTa-RP350k-spv")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-RP350k-spv/commit/153f8102e20dab8b22f3d64b88febb32f5c4492d', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='153f8102e20dab8b22f3d64b88febb32f5c4492d', pr_url=None, pr_revision=None, pr_num=None)

### WVMCQ7

In [ ]:
df_wvs_binary = df.progress_apply(to_binary,axis=1)

  0%|          | 0/15228 [00:00<?, ?it/s]

In [ ]:
df_wvs_binary['input_ids'] = 0
df_wvs_binary['attention_mask'] = 0

In [ ]:
df_wvs_binary['split']=df_wvs_binary.input.map(lambda x: get_split(x,0.90,0.10))
df_wvs_binary=DatasetDict({k:datasets.Dataset.from_pandas(df_wvs_binary[df_wvs_binary['split']==k]) for k in {'train','test','validation'}})

In [ ]:
from xpflow import Xp
lr = 1e-5
bs = 8
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=10
      logging_strategy = "steps"
      warmup_ratio=0.06
      loss = ["torch.nn.CrossEntropyLoss()"]
      gradient_accumulation_steps = 8


reco = tn.MultipleChoice(
    df_wvs_binary,
    s1="question",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
)

tasks = [reco]

for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()
  wandb.finish()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 8 ---------------------------- 
 


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Name,Size,Index
0,0.781400,1.081891,0.583815,0.249977,,1903,0
1,0.622800,1.437966,0.528639,0.256194,,1903,0
2,0.577600,1.135929,0.538623,0.289160,,1903,0
3,0.538700,1.455635,0.528114,0.287344,,1903,0
4,0.566200,1.452054,0.512874,0.252577,,1903,0
5,0.519700,1.711763,0.560168,0.303403,,1903,0
6,0.490500,1.454695,0.564898,0.301330,,1903,0
8,0.473200,1.487321,0.554388,0.296864,,1903,0
8,0.473200,1.602946,0.555439,0.298322,,1903,0
9,0.430700,1.740879,0.544404,0.277999,,1903,0


Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a

/epoch,▁▂▃▃▄▅▆▆▇█
/eval_accuracy,█▃▄▃▁▆▆▅▅▄
/eval_f1,▁▂▆▆▁██▇▇▅
/eval_index,▁▁▁▁▁▁▁▁▁▁
/eval_loss,▁▅▂▅▅█▅▅▇█
/eval_runtime,█▄▃▁▂▃▂▄▃▄
/eval_samples_per_second,▁▄▆█▇▆▇▅▆▅
/eval_size,▁▁▁▁▁▁▁▁▁▁
/eval_steps_per_second,▁▄▆█▇▆▇▅▆▅
epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
gradient_accumulation_steps,▁


In [ ]:
model, adapter = trainer.model.factorize()
model.push_to_hub("Qubix/RoBERTa-WVMCQ7-Classification-CE")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Qubix/RoBERTa-WVMCQ7-Classification-CE/commit/67133066dba732237d0774d14907426f2e1d8e67', commit_message='Upload RobertaForMultipleChoice', commit_description='', oid='67133066dba732237d0774d14907426f2e1d8e67', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
pd.DataFrame(dataset_reddit_binary_sign["validation"]).to_json("validation_set_WVMCQ7_CE_model.json")

## Regression

In [110]:
df_reddit_sign = pd.DataFrame([item  for item in df_reddit_demographics_normalized_labels.values if item[1] in redditPoll350k_significant_reduced["question"].values],columns = df_reddit_demographics_normalized_labels.columns)

In [111]:
df_reddit_sign["labels"] = df_reddit_sign["label"]

In [112]:
df_reddit_sign['input_ids'] = 0
df_reddit_sign['attention_mask'] = 0

In [113]:
df_reddit_sign =  pd.DataFrame([item for item in df_reddit_sign.values if df_reddit_sign["demographic"].value_counts()[item[6]]>49],columns=df_reddit_sign.columns)
df_reddit_sign_undersampled = df_reddit_sign.undersample(column="demographic")

In [114]:
df_reddit_sign_undersampled["inputs"] = df_reddit_sign_undersampled["demographic"] +": "+df_reddit_sign_undersampled["title"]
df_reddit_sign["inputs"] = df_reddit_sign["demographic"] +": "+df_reddit_sign["title"]

In [115]:
df_reddit_sign['split']=df_reddit_sign.title.map(lambda x: get_split(x,0.90,0.10))
dataset_reddit_binary_sign=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_sign[df_reddit_sign['split']==k]) for k in {'train','test','validation'}})
dataset_reddit_binary_sign

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 1145
    })
    validation: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 123
    })
    test: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 0
    })
})

In [128]:
def get_split(x):
    """parameters are wished percentage of dataset to be given to a split"""
    rnd=random.Random(x).random()
    if rnd<0.90:
        return 'train'
    if rnd<(1):
        return 'validation'
    else:
        return 'test'

In [129]:
df_reddit_sign_undersampled['split']=df_reddit_sign_undersampled.title.map(lambda x: get_split(x))
dataset_reddit_sign=DatasetDict({k:datasets.Dataset.from_pandas(df_reddit_sign_undersampled[df_reddit_sign_undersampled['split']==k]) for k in {'train','test','validation'}})
dataset_reddit_sign

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 311
    })
    validation: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 29
    })
    test: Dataset({
        features: ['id', 'title', 'created', 'inputs', 'n_answers', 'split', 'demographic', 'choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 'label', 'task', 'labels', 'input_ids', 'attention_mask', '__index_level_0__'],
        num_rows: 0
    })
})

In [ ]:
from xpflow import Xp
lr = 1e-5
bs = 8
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=20
      batch_size =bs
      loss = ["torch.nn.CrossEntropyLoss()"]
      gradient_accumulation_steps = 1#dont need this as batch already makes batches of 8 examples

reco = tn.MultipleChoice(
    dataset_reddit_binary_sign,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
)
for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.batch_size)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()#NOW
  wandb.finish()

In [143]:
from xpflow import Xp
lr = 1e-5
bs = 8
class cv(Xp):
      model_name = "roberta-base"
      # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
      learning_rate = lr
      num_train_epochs=20
      batch_size =bs
      logging_strategy = "steps"
      warmup_ratio=0.06
      loss = ['torch.nn.BCELoss()','mse_loss'][0]
      gradient_accumulation_steps = 1



reco = tn.MultipleChoice(
    dataset_reddit_sign,
    s1="inputs",
    y="labels",
    num_labels = num_choices,
    choices=choices_names,
    data_collator=DataCollatorForPolls()
)

tasks = [reco]

for args in cv():
  wandb.init(project="polls", entity="quentintilman")
  wandb.log(args)
  tasks = [reco]
  models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head 
  models.task_models_list[0].loss = eval(args.loss)
  models.task_models_list[0].forward = MethodType(forward, models.task_models_list[0])
  tasks[0].compute_metrics=MethodType(compute_metrics_bce,tasks[0])
  trainer = tn.Trainer(models, tasks, args,tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)) # tasks are uniformly sampled by default
  print("\n ------------------------------ START "+str(args.learning_rate)+" & "+str(args.gradient_accumulation_steps)+" ---------------------------- \n ")
  trainer.evaluate()
  trainer.train()
  wandb.finish()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



 ------------------------------ START 1e-05 & 1 ---------------------------- 
 


  0%|          | 0/29 [00:00<?, ?it/s]

Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


  0%|          | 0/6220 [00:00<?, ?it/s]

...

{'eval_loss': 0.2550695836544037, 'eval_name': '', 'eval_size': 29, 'eval_index': 0, 'eval_distance': 0.2554870545864105, 'eval_runtime': 1.4316, 'eval_samples_per_second': 20.257, 'eval_steps_per_second': 2.794, 'epoch': 19.0}
{'loss': 0.2382, 'learning_rate': 3.7632569278138903e-07, 'epoch': 19.29}


  0%|          | 0/29 [00:00<?, ?it/s]

Trainer is attempting to log a value of "" of type <class 'str'> for key "eval/name" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.26987123489379883, 'eval_name': '', 'eval_size': 29, 'eval_index': 0, 'eval_distance': 0.2554647624492645, 'eval_runtime': 1.4107, 'eval_samples_per_second': 20.557, 'eval_steps_per_second': 2.835, 'epoch': 20.0}
{'train_runtime': 1145.9974, 'train_samples_per_second': 5.428, 'train_steps_per_second': 5.428, 'train_loss': 0.23855478403269287, 'epoch': 20.0}


/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
/eval_distance,▁▇████████████▇█▇▇▇▆
/eval_index,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
/eval_loss,▇▄▁▃█▇▅█▇▅█▅▁▅█▁▁▇▅▇
/eval_runtime,▂▂▁▄▆▃▇▅▄▆▆▇▄██▅▆▅▃▁
/eval_samples_per_second,▇▇█▅▃▆▂▄▅▃▂▂▅▁▁▄▃▄▆█
/eval_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
/eval_steps_per_second,▇▇█▅▃▆▂▄▅▃▂▂▅▁▁▄▃▄▆█
batch_size,▁
epoch,▁▂▂▃▃▄▅▅▆▇▇██
gradient_accumulation_steps,▁


## Baseline in classification Task

In [102]:
classifier = transformers.pipeline("fill-mask",device= torch.device("cpu"),model="roberta-base")

In [103]:
def softmax(x):
   return np.exp(x)/sum(np.exp(x))

In [108]:
from decimal import *
def perform_zeroshot_demographic(x):
  title = x["title"]
  year = x["created"]
  label = x["labels"]
  demographic = x["demographic"]
  question_input = x["title"]
  choices = [x[f'choice_{i}'] for i in range((sum([1 for l in label if l != -1])))]
  zeroshot_preds = []
  try:
    for choice in choices:
      zeroshot_preds.append(round((int(classifier(f"In {year}, <mask> percent of {demographic} voted '{choice}' on the question '{question_input}'".lower())[0]["token_str"]) / 100),2))
    x["zeroshot_preds"] = np.argmax(np.array(softmax(zeroshot_preds)))
    x["cat_label"] = np.argmax(x["labels"])
  except:
    pass
  return x

In [ ]:
validation_set_sgn_reddit = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Experiments/validation_set_sgn_reddit.json",lines=True)

In [110]:
validation_set_sgn_reddit = validation_set_sgn_reddit.progress_apply(perform_zeroshot_demographic,axis=1)

  0%|          | 0/68 [00:00<?, ?it/s]

In [120]:
from sklearn.metrics import f1_score, accuracy_score
y_true = validation_set_sgn_reddit["cat_label"].values
y_pred = validation_set_sgn_reddit["zeroshot_preds"].values
f1 = f1_score(y_true,y_pred,average='macro')
acc = accuracy_score(y_true,y_pred)
print(f'F1-score: {f1}\n Accuracy: {acc}')

F1-score: 0.2307692307692308
 Accuracy: 0.5294117647058824


In [125]:
rp350k_sgn = pd.read_json("https://raw.githubusercontent.com/QuentinTilman/PollBERT-paper/master/Datasets/RedditPoll350K/RedditPoll350K_significant_divisiveness_questions.json")

In [131]:
df_reddit_demographics_normalized_labels[df_reddit_demographics_normalized_labels["title"] == "Do you watch New Year’s Eve Drop Ball on Live TV?"]

,id,title,created,inputs,n_answers,split,demographic,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,label,task
265829,klzj84,Do you watch New Year’s Eve Drop Ball on Live TV?,1.609191e+09,american: Do you watch New Year’s Eve Drop Bal...,125.0,True,american,Yes,No,,,,,"[0.45348837209302323, 0.5465116279069767, -1.0...",0.0
265829,klzj84,Do you watch New Year’s Eve Drop Ball on Live TV?,1.609191e+09,not american: Do you watch New Year’s Eve Drop...,125.0,True,not american,Yes,No,,,,,"[0.1282051282051282, 0.8717948717948718, -1.0,...",0.0
